# Install required package

In [1]:
%pip install gurobipy

Note: you may need to restart the kernel to use updated packages.


# Import required packages

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
import ws3.forest, ws3.core
import gurobipy as grb
import csv
import numpy as np
import gurobipy as grb

# Set up python environment

In [4]:
# Defining basic parameters
base_year = 2020
horizon = 10
period_length = 10
max_age =  1000
tvy_name = 'totvol'

In [5]:
# Reading stands file
# stands = gpd.read_file('data/tsa04.shp/stands selection.shp') #Golden Bear
stands = gpd.read_file('data/tsa04contclass/Resultand_GoldenBear_v2 selection.shp') #contclass

In [6]:
stands

FID_Golden  FID_Gold_1  ZONE SUBZONE VARIANT PHASE NTRLDSTRBN  \
0               1           1   SWB     uns    None  None       NDT2   
1               1           1   SWB     uns    None  None       NDT2   
2               1           1   SWB     uns    None  None       NDT2   
3               1           1   SWB     uns    None  None       NDT2   
4               1           1   SWB     uns    None  None       NDT2   
...           ...         ...   ...     ...     ...   ...        ...   
63203           1           7  BWBS      dk    None  None       NDT3   
63204           1           7  BWBS      dk    None  None       NDT3   
63205           1           7  BWBS      dk    None  None       NDT3   
63206           1          11    MH      mm       2  None       NDT1   
63207           1          11    MH      mm       2  None       NDT1   

      MAP_LABEL  FID_Gold_2 HERD_STATU  ... ORIG_FID   Shape_Leng  \
0        SWBuns          -1       None  ...      468   892.970730   
1        SWBuns          -1       None  ...      468   847.015427   
2        SWBuns          -1       None  ...      468  2146.240646   
3        SWBuns          -1       None  ...      468   255.813594   
4        SWBuns          -1       None  ...      469   859.337139   
...         ...         ...        ...  ...      ...          ...   
63203    BWBSdk          -1       None  ...    71281   326.907495   
63204    BWBSdk          -1       None  ...    71281   476.065471   
63205    BWBSdk          -1       None  ...    71281   415.325791   
63206     MHmm2          -1       None  ...    71292   287.561926   
63207     MHmm2          -1       None  ...    71293   286.917119   

         Shape_Area contclass      rollup             netdown  THLB_Area  \
0      16474.221678         C        THLB                THLB   1.647422   
1      25214.104031         C        THLB                THLB   2.521410   
2      87035.420516         C        THLB                THLB   8.703542   
3       2305.734220         C        THLB                THLB   0.230573   
4      15510.057930         C        THLB                THLB   1.551006   
...             ...       ...         ...                 ...        ...   
63203   1502.856584         N  6_Riparian  6_01_Stream_Buffer   0.000000   
63204   1950.527358         N  6_Riparian  6_01_Stream_Buffer   0.000000   
63205   1606.933608         N  6_Riparian  6_01_Stream_Buffer   0.000000   
63206   1279.447945         N  6_Riparian  6_01_Stream_Buffer   0.000000   
63207   1269.538029         N  6_Riparian  6_01_Stream_Buffer   0.000000   

      Age_2023  AU                                           geometry  
0           46   6  POLYGON ((651330.000 1473337.500, 651330.000 1...  
1           46   6  POLYGON ((651251.113 1473060.000, 651250.257 1...  
2           46   6  POLYGON ((651090.000 1473000.000, 651090.000 1...  
3           46   6  POLYGON ((650746.974 1472340.000, 650640.000 1...  
4           46   6  POLYGON ((651390.000 1473330.000, 651390.000 1...  
...        ...  ..                                                ...  
63203       83  33  POLYGON ((649003.440 1506715.961, 649006.877 1...  
63204       83  33  POLYGON ((648270.000 1506662.850, 648270.000 1...  
63205       83  33  POLYGON ((649050.000 1506509.390, 649050.000 1...  
63206      101  33  POLYGON ((622490.804 1495979.904, 622468.035 1...  
63207      101  33  POLYGON ((622510.975 1495990.427, 622490.804 1...  

[63208 rows x 244 columns]

In [7]:
stands = stands.rename(columns={'thlb': 'theme1', 'au': 'theme2', 'ldspp': 'theme3', 'age2015': 'age', 'shape_area': 'area' })

In [8]:
# Fixing areas using geometry data (initially all areas were zero)
stands['area'] = stands.geometry.area*0.0001

In [9]:
# Adding theme 4 (which is curve_id) to the stands table
stands.insert(4, 'theme4', stands['theme2'])

KeyError: 'theme2'

In [ ]:
# This codes aims to convert the stand file into a shp file for showing on ArcGis Pro to checks the shape of polygons
stands.to_file('CHECK_SHP')

##### Modified for TSA 17 (ECCC projetc) to extract the unique AU from yld.csv file 

In [ ]:
# This code changes the type of "theme2" into integer
stands['theme2'] = stands['theme2'].astype(int)

In [ ]:
# Reading yld table and changing AU column type into integer
yld = pd.read_csv('data/yld.csv')
yld['AU'] = yld['AU'].astype(int)

In [ ]:
# Creating AU table from stands table and renaming the column to AU. The AU table is then joined with yld table.
AU = pd.DataFrame(stands['theme2']).drop_duplicates()
AU.rename(columns = {'theme2':'AU'},inplace=True)

In [ ]:
# Inner joining AU and yld tables. The results of the code will be a yield table (yldmerged) that has AU column.
yldmerged = pd.merge(AU, yld, on=['AU'], how='inner')

In [ ]:
# Getting names and codes of canfi_species. This table will be used to add canfi_species column into yldmerged
canf = pd.read_csv('data/canfi_species_modified.csv')
canf = canf[['name','canfi_species']].set_index('name')

In [ ]:
# Adding codes of leading species under canfi_species in the yldmerged
for index, row in yldmerged.iterrows():
    yldmerged.at[index,'canfi_species'] = canf.loc[row['LDSPP'],'canfi_species']  

In [ ]:
# once the modifications are done, the yldmerged table is converted into csv file in a directory path defined by the user. Later, this file will be used to read curve_points_table
output_csv_path = 'data/yldmerged.csv'
yldmerged.to_csv(output_csv_path, header=True, index=False)

##### End of modification 

In [ ]:
# converting stands to csv file, and reading stands_table from the csv file
output_stand_csv_path = 'data/stands_table.csv'
stands_table = stands.to_csv(output_stand_csv_path, header=True, index=False)
stands_table = pd.read_csv('data/stands_table.csv')

In [ ]:
# replacing leading species names with their codes
for index, row in stands_table.iterrows():
    stands_table.at[index,'theme5'] = canf.loc[row['theme3'],'canfi_species']
stands_table['theme3']=stands_table['theme5']
stands_table=stands_table.drop('theme5',axis=1)


In [ ]:
# Reading curve_points_table
curve_points_table = pd.read_csv('data/yldmerged.csv')

In [ ]:
# Adding 'curve_id' and 'canfi_species' columns to curve_points_table
curve_points_table ['curve_id'] = curve_points_table ['AU'] 
curve_points_table ['canfi_species'] = yldmerged ['canfi_species']

In [ ]:
# converting curve_points_table to csv file
output_curve_points_csv_path = 'data/curve_points_table.csv'
curve_points_table.to_csv(output_curve_points_csv_path, header=True, index=False)

In [ ]:
# Reading curve_points_table from the csv file and settign AU as the index
columns_to_index_yld = ['AU']
curve_points_table = pd.read_csv('data/curve_points_table.csv').set_index(columns_to_index_yld)

In [ ]:
# Setting up themes
theme_cols=['theme0', # TSA 
            'theme1', # THLB
            'theme2', # AU
            'theme3', # leading species code
            'theme4'  # yield curve ID
           ]
basecodes = [list(map(lambda x: str(x), stands_table[tc].unique())) for tc in theme_cols]

In [ ]:
# also scrape au_table for AU and curve ID values that are not in inventory but might pop up later (hack?)
basecodes[2] = list(set(basecodes[2] + list(stands_table['theme2'].astype(str))))
basecodes[3] = list(set(basecodes[3] + list(stands_table['theme3'].astype(str))))
basecodes[4] = list(set(basecodes[4] + list(stands_table['theme4'].astype(str))))

In [ ]:
# Building the forest model
fm = ws3.forest.ForestModel(model_name='tsa04',
                            model_path='data',
                            base_year=base_year,
                            horizon=horizon,
                            period_length=period_length,
                            max_age=max_age)

In [ ]:
# Set up themes
for ti, t in enumerate(theme_cols):
    fm.add_theme(t, basecodes[ti])
fm.nthemes = len(theme_cols)

In [ ]:
# Aggregating inventory data
gstands = stands_table.groupby(theme_cols+['age'])

In [ ]:
# Loading areas from inventory
age_cutoff = 600

for name, group in gstands:
    dtk, age, area = tuple(map(lambda x: str(x), name[:-1])), int(name[-1]), group['area'].sum()
    if dtk not in fm.dtypes:
        fm.dtypes[dtk] = ws3.forest.DevelopmentType(dtk, fm)
    age = age if age < age_cutoff else age_cutoff
    fm.dtypes[dtk].area(0, age, area)

In [ ]:
# Reading points_table which was created in csv file in order to convert to a 2-column table (x,y)
points_table = pd.read_csv('data/points_table.csv').set_index('AU')

In [ ]:
# Reading and registering curve points
for AU, au_row in curve_points_table.iterrows():
    yname = 's%04d' % int(au_row.canfi_species)    
    curve_id = au_row.curve_id
    mask = ('?', '?', str(AU), '?', str(curve_id))
    points = [(r.x, r.y) for _, r in points_table.loc[AU].iterrows() if not r.x % period_length and r.x <= max_age]    
    c = fm.register_curve(ws3.core.Curve(yname, points=points, type='a', is_volume=True, xmax=fm.max_age, period_length=period_length))
    fm.yields.append((mask, 'a', [(yname, c)])) # only if not already present?
    fm.ynames.add(yname)        
    for dtk in fm.unmask(mask): 
        fm.dtypes[dtk].add_ycomp('a', yname, c)

In [ ]:
# Add total volume curves
expr = '_SUM(%s)' % ', '.join(fm.ynames)
fm.yields.append((('?', '?', '?', '?', '?'), 'c', [(tvy_name, expr)]))
fm.ynames.add(tvy_name)
for dtk in fm.dtypes.keys(): fm.dtypes[dtk].add_ycomp('c', tvy_name, expr)  

In [ ]:
# Setting up an action
acode = 'harvest'
# oe = '_age >= 40 and _age <= 500' # operability expression
oe = '_age >= 100 and _age <= 600' # operability expression
fm.transitions[acode] = {}
for au_id, au_row in stands_table.iterrows():
    if not au_row.theme1: continue
    target_curve_id = au_row.theme4  
    smask = ('?', '1' , str (target_curve_id), '?', '?')
    tmask = ('?', '1' , '?', '?', str(target_curve_id) ) 
    target = [(tmask, 1.0, None, None, None, None, None)] # list of one (single target... not modelling "divergent" transitions)
    fm.actions[acode] = ws3.forest.Action(acode, targetage=0, is_harvest=True)
    fm.oper_expr[acode] = {smask:oe}        
    fm.transitions[acode].update({smask:{'':target}}) # the '' is a blank source condition expression
    for dtk in fm.unmask(smask):
        dt = fm.dtypes[dtk]
        dt.oper_expr[acode] = [oe]
        for age in range(1, fm.max_age):
            if not dt.is_operable(acode, 1, age): continue          
            fm.dtypes[dtk].transitions[acode, age] = target

In [ ]:
fm.compile_actions()

# Poking around the model

In [ ]:
fig, ax = plt.subplots(3, 1, figsize=(8, 12), sharex=True)

cvol = c
ccai = c.cai()
cmai = c.mai()
cmaiytp = c.mai().ytp()
x_cmai = cmaiytp.lookup(0) # optimal rotation age (i.e., culmination of MAI curve)
labels = 'total volume', 'MAI (and CAI)', 'MAI YTP'

ax[0].plot(*zip(*c.points()))
ax[0].plot([0, x_cmai], [0., cvol[x_cmai]], linestyle='--', color='green')

ax[1].plot(*zip(*c.mai().points()))
ax[1].plot(*zip(*c.cai().points()), linestyle=':')

ax[2].plot(*zip(*c.mai().ytp().points()))
ax[2].axhline(0, color='black')

for i in range(len(ax)):
    ax[i].set_ylabel(labels[i])
    ax[i].set_ylim(0, None)
    ax[i].axvline(x_cmai, color='red')
plt.xlim(0, 350)

In [ ]:
import matplotlib.pyplot as plt

# Assuming fm.age_class_distribution(0) returns a dictionary or two lists (x and y)
data = fm.age_class_distribution(0)

# Extracting x and y values
x_values = data.keys()  # replace with the actual method to get x values
y_values = data.values()  # replace with the actual method to get y values

# Create a bar chart
plt.bar(x_values, y_values, color='blue', edgecolor='black')

# Add labels and title
plt.xlabel('Age Class')
plt.ylabel('Frequency')
plt.title('Age Class Distribution')

# Show the plot
plt.show()


# Implement a priority queue harvest scheduling heuristic


In [ ]:
def schedule_harvest_areacontrol(fm, period=1, acode='harvest', util=0.85, 
                                 target_masks=None, target_areas=None, target_scalefactors=None,
                                 mask_area_thresh=0.,
                                 verbose=0):
    if not target_areas:
        if not target_masks: # default to AU-wise THLB 
            au_vals = []
            au_agg = []
            for au in fm.theme_basecodes(2):
                mask = '? 1 %s ? ?' % au
                masked_area = fm.inventory(0, mask=mask)
                if masked_area > mask_area_thresh:
                    au_vals.append(au)
                else:
                    au_agg.append(au)
                    if verbose > 0:
                        print('adding to au_agg', mask, masked_area)
            if au_agg:
                fm._themes[2]['areacontrol_au_agg'] = au_agg 
                if fm.inventory(0, mask='? ? areacontrol_au_agg ? ?') > mask_area_thresh:
                    au_vals.append('areacontrol_au_agg')
            target_masks = ['? 1 %s ? ?' % au for au in au_vals]
        target_areas = []
        for i, mask in enumerate(target_masks): # compute area-weighted mean CMAI age for each masked DT set
            masked_area = fm.inventory(0, mask=mask, verbose=verbose)
            if not masked_area: continue
            r = sum((fm.dtypes[dtk].ycomp('totvol').mai().ytp().lookup(0) * fm.dtypes[dtk].area(0)) for dtk in fm.unmask(mask))
            r /= masked_area
            asf = 1. if not target_scalefactors else target_scalefactors[i]  
            ta = (1/r) * fm.period_length * masked_area * asf
            target_areas.append(ta)
    for mask, target_area in zip(target_masks, target_areas):
        if verbose > 0:
            print('calling areaselector', period, acode, target_area, mask)
        fm.areaselector.operate(period, acode, target_area, mask=mask, verbose=verbose)
    sch = fm.compile_schedule()
    return sch

In [ ]:
def plot_results(fm):
    pareas = [fm.compile_product(period, '1.') for period in fm.periods]
    pvols = [fm.compile_product(period, 'totvol') for period in fm.periods]
    df = pd.DataFrame({'period':fm.periods, 'ha':pareas, 'hv':pvols})
    fig, ax = plt.subplots(3, 1, figsize=(8, 12), sharex=True)
    ax[0].set_ylabel('harvest area')
    ax[0].bar(df.period, df.ha)
    ax[1].set_ylabel('harvest volume')
    ax[1].bar(df.period, df.hv)
    ax[2].set_ylabel('harvest volume:area ratio')
    ax[2].bar(df.period, (df.hv/df.ha).fillna(0))
    ax[2].set_ylim(0, None)
    return fig, ax, df

In [ ]:
fm.reset()

In [ ]:
verbose = False # flip to True if you want the harvest scheduler to get _really_ chatty while it works
for period in fm.periods:
    schedule_harvest_areacontrol(fm, period=period, verbose=1)

In [ ]:
pareas = [fm.compile_product(period, '1.') for period in fm.periods]

In [ ]:
pvols = [fm.compile_product(period, 'totvol') for period in fm.periods]

In [ ]:
df = pd.DataFrame({'period':fm.periods, 'areas':pareas, 'vols':pvols})
df

In [ ]:
plot_results(fm)

# Implement optimization-based action scheduling


In [ ]:
# %pip install gurobipy

In [ ]:
# import gurobipy as grb

In [ ]:
def cmp_c_z(fm, path, expr):
    """
    Compile objective function coefficient (given ForestModel instance, 
    leaf-to-root-node path, and expression to evaluate).
    """
    result = 0.
    for t, n in enumerate(path, start=1):
        d = n.data()
        if fm.is_harvest(d['acode']):
            result += fm.compile_product(t, expr, d['acode'], [d['dtk']], d['age'], coeff=False)
    return result

def cmp_c_cflw(fm, path, expr, mask=None): # product, all harvest actions
    """
    Compile flow constraint coefficient for product indicator (given ForestModel 
    instance, leaf-to-root-node path, expression to evaluate, and optional mask).
    """
    result = {}
    for t, n in enumerate(path, start=1):
        d = n.data()
        if mask and not fm.match_mask(mask, d['dtk']): continue
        if fm.is_harvest(d['acode']):
            result[t] = fm.compile_product(t, expr, d['acode'], [d['dtk']], d['age'], coeff=False)
    return result


def cmp_c_caa(fm, path, expr, acodes, mask=None): # product, named actions
    """
    Compile constraint coefficient for product indicator (given ForestModel 
    instance, leaf-to-root-node path, expression to evaluate, list of action codes, 
    and optional mask).
    """
    result = {}
    for t, n in enumerate(path, start=1):
        d = n.data()
        if mask and not fm.match_mask(mask, d['dtk']): continue
        if d['acode'] in acodes:
            result[t] = fm.compile_product(t, expr, d['acode'], [d['dtk']], d['age'], coeff=False)
    return result


def cmp_c_ci(fm, path, yname, mask=None): # product, named actions
    """
    Compile constraint coefficient for inventory indicator (given ForestModel instance, 
    leaf-to-root-node path, expression to evaluate, and optional mask).
    """
    result = {}
    for t, n in enumerate(path, start=1):
        d = n.data()
        if mask and not fm.match_mask(mask, d['_dtk']): continue
        result[t] = fm.inventory(t, yname=yname, age=d['_age'], dtype_keys=[d['_dtk']]) 
        #result[t] = fm.inventory(t, yname=yname, age=d['age'], dtype_keys=[d['dtk']]) 
    return result

In [ ]:
def gen_scenario(fm, name='base', util=0.85, harvest_acode='harvest',
                 cflw_ha={}, cflw_hv={}, 
                 cgen_ha={}, cgen_hv={}, cgen_gs={}, 
                 tvy_name='totvol', obj_mode='max_hv', mask=None):
    from functools import partial
    import numpy as np
    coeff_funcs = {}
    cflw_e = {}
    cgen_data = {}
    acodes = ['null', harvest_acode] # define list of action codes
    vexpr = '%s * %0.2f' % (tvy_name, util) # define volume expression
    if obj_mode == 'max_hv': # maximize harvest volume
        sense = ws3.opt.SENSE_MAXIMIZE 
        zexpr = vexpr
    elif obj_mode == 'min_ha': # minimize harvest area
        sense = ws3.opt.SENSE_MINIMIZE 
        zexpr = '1.'
    else:
        raise ValueError('Invalid obj_mode: %s' % obj_mode)        
    coeff_funcs['z'] = partial(cmp_c_z, expr=zexpr) # define objective function coefficient function  
    T = fm.periods
    if cflw_ha: # define even flow constraint (on harvest area)
        cname = 'cflw_ha'
        coeff_funcs[cname] = partial(cmp_c_caa, expr='1.', acodes=[harvest_acode], mask=None) 
        cflw_e[cname] = cflw_ha
    if cflw_hv: # define even flow constraint (on harvest volume)
        cname = 'cflw_hv'
        coeff_funcs[cname] = partial(cmp_c_caa, expr=vexpr, acodes=[harvest_acode], mask=None) 
        cflw_e[cname] = cflw_hv         
    if cgen_ha: # define general constraint (harvest area)
        cname = 'cgen_ha'
        coeff_funcs[cname] = partial(cmp_c_caa, expr='1.', acodes=[harvest_acode], mask=None) 
        cgen_data[cname] = cgen_ha
    if cgen_hv: # define general constraint (harvest volume)
        cname = 'cgen_hv'
        coeff_funcs[cname] = partial(cmp_c_caa, expr=vexpr, acodes=[harvest_acode], mask=None) 
        cgen_data[cname] = cgen_hv
    if cgen_gs: # define general constraint (growing stock)
        cname = 'cgen_gs'
        coeff_funcs[cname] = partial(cmp_c_ci, yname=tvy_name, mask=None)
        cgen_data[cname] = cgen_gs
    return fm.add_problem(name, coeff_funcs, cflw_e, cgen_data=cgen_data, acodes=acodes, sense=sense, mask=mask)

In [ ]:
def compile_scenario(fm):
    oha = [fm.compile_product(period, '1.', acode='harvest') for period in fm.periods]
    ohv = [fm.compile_product(period, 'totvol * 0.85', acode='harvest') for period in fm.periods]
    ogs = [fm.inventory(period, 'totvol') for period in fm.periods]
    data = {'period':fm.periods, 
            'oha':oha, 
            'ohv':ohv, 
            'ogs':ogs}
    df = pd.DataFrame(data)
    return df

In [ ]:
def plot_scenario(df):
    fig, ax = plt.subplots(1, 3, figsize=(12, 4))
    ax[0].bar(df.period, df.oha)
    ax[0].set_ylim(0, None)
    ax[0].set_title('Harvested area (ha)')
    ax[1].bar(df.period, df.ohv)
    ax[1].set_ylim(0, None)
    ax[1].set_title('Harvested volume (m3)')
    ax[2].bar(df.period, df.ogs)
    ax[2].set_ylim(0, None)
    ax[2].set_title('Growing Stock (m3)')
    return fig, ax

In [ ]:
fm.add_null_action()

In [ ]:
def run_scenario(fm, scenario_name='base'):
    cflw_ha = {}
    cflw_hv = {}
    cgen_ha = {}
    cgen_hv = {}
    cgen_gs = {}
    
    # define harvest area and harvest volume flow constraints
    cflw_ha = ({p:0.05 for p in fm.periods}, 1)
    cflw_hv = ({p:0.05 for p in fm.periods}, 1)

    if scenario_name == 'base': 
        # Base scenario
        print('running bsae scenario')
    elif scenario_name == 'base-cgen_ha': 
        # Base scenario, plus harvest area general constraints
        print('running base scenario plus harvest area constraints')
        cgen_ha = {'lb':{1:100.}, 'ub':{1:101.}}    
    elif scenario_name == 'base-cgen_hv': 
        # Base scenario, plus harvest volume general constraints
        print('running base scenario plus harvest volume constraints')
        cgen_hv = {'lb':{1:1000.}, 'ub':{1:1001.}}    
    elif scenario_name == 'base-cgen_gs': 
        # Base scenario, plus growing stock general constraints
        print('running base scenario plus growing stock constraints')
        cgen_gs = {'lb':{10:100000.}, 'ub':{10:100001.}}
    else:
        assert False # bad scenario name

    p = gen_scenario(fm=fm, 
                     name=scenario_name, 
                     cflw_ha=cflw_ha, 
                     cflw_hv=cflw_hv,
                     cgen_ha=cgen_ha,
                     cgen_hv=cgen_hv,
                     cgen_gs=cgen_gs)

    fm.reset()
    m = p.solve()

    if m.status != grb.GRB.OPTIMAL:
        print('Model not optimal.')
        sys.exit()
    sch = fm.compile_schedule(p)
    fm.apply_schedule(sch, 
                      force_integral_area=False, 
                      override_operability=False,
                      fuzzy_age=False,
                      recourse_enabled=False,
                      verbose=False,
                      compile_c_ycomps=True)
    df = compile_scenario(fm)
    fig, ax = plot_scenario(df)
    return fig, df, p

In [ ]:
run_scenario(fm, 'base')

In [ ]:
fig, df, problem = run_scenario(fm, 'base-cgen_ha')

In [ ]:
run_scenario(fm, 'base-cgen_hv')

In [ ]:
run_scenario(fm, 'base-cgen_gs')

In [ ]:
!mkdir data/woodstock_model_files

# LANDSCAPE section

In [ ]:
fm._theme_basecodes

In [ ]:
for v in fm._theme_basecodes[2]: print(v)


In [ ]:
for v in fm._theme_basecodes[3]: print(v)

In [ ]:
for v in fm._theme_basecodes[4]: print(v)

In [ ]:
landscape_section = \
"""
*THEME Timber Supply Area (TSA)
tsa04

*THEME Timber Harvesting Land Base (THLB)
0
1

*THEME Analysis Unit (AU)
505
546
74
514
515
170
78
201
267
256
767
265
536
124
762
689
61
298
1203
728
509
79
733
756
698
54
306
1085
60
81
1191
548
1026


*THEME Leading tree species (CANFI species code)
101.0
402.0
1200.0
304.0
100.0
204.0
403.0
301.0
302.0
1211.0

*THEME Yield curve ID
505
546
74
514
515
170
78
201
267
256
767
265
536
124
762
689
61
298
1203
728
509
79
733
756
698
54
306
1085
60
81
1191
548
1026
"""

In [ ]:
with open('data/woodstock_model_files/tsa04.lan', 'w') as f: f.write(landscape_section)

# AREAS section

In [ ]:
for name, group in gstands:
    dtk, age, area = tuple(map(lambda x: str(x), name[:-1])), int(name[-1]), group['area'].sum()
    print('*A', ' '.join(v for v in dtk), age, area)

In [ ]:
areas_section = \
"""
*A tsa04 0 54 1211.0 54 70 212.0773035402646
*A tsa04 0 54 1211.0 54 74 30.404265767023357
*A tsa04 0 54 1211.0 54 80 786.8129868835314
*A tsa04 0 54 1211.0 54 90 4866.282625540163
*A tsa04 0 54 1211.0 54 110 124.18132996976581
*A tsa04 0 60 304.0 60 90 431.6918652452945
*A tsa04 0 60 304.0 60 174 55.00641005877971
*A tsa04 0 60 304.0 60 220 58.37668474603872
*A tsa04 0 60 304.0 60 240 640.0027445438932
*A tsa04 0 61 304.0 61 64 91.06955990382784
*A tsa04 0 61 304.0 61 74 177.37583453659832
*A tsa04 0 61 304.0 61 240 245.14875345640618
*A tsa04 0 74 204.0 74 36 12.265771622767792
*A tsa04 0 74 204.0 74 90 997.8136449928539
*A tsa04 0 74 204.0 74 110 70.73446729339769
*A tsa04 0 74 204.0 74 130 396.0955780940215
*A tsa04 0 74 204.0 74 150 160.9053750048491
*A tsa04 0 74 204.0 74 240 57.437669189547286
*A tsa04 0 78 100.0 78 54 27.914763514402992
*A tsa04 0 78 100.0 78 110 157.79248423440177
*A tsa04 0 78 100.0 78 130 79.93065721129835
*A tsa04 0 78 100.0 78 150 101.68769539629616
*A tsa04 0 78 100.0 78 170 45.062365558859625
*A tsa04 0 78 100.0 78 174 5.2837358001161165
*A tsa04 0 78 100.0 78 225 24.720458108627227
*A tsa04 0 78 100.0 78 240 2255.573111749515
*A tsa04 0 79 100.0 79 90 188.5850336691844
*A tsa04 0 81 101.0 81 240 137.56695220151408
*A tsa04 0 124 302.0 124 240 13.02390390932942
*A tsa04 0 170 301.0 170 342 35.66001038833771
*A tsa04 0 201 402.0 201 342 42.834595822627804
*A tsa04 0 256 1211.0 256 90 2309.481572340572
*A tsa04 0 256 1211.0 256 110 72.41377268875031
*A tsa04 0 265 304.0 265 49 275.2516325158967
*A tsa04 0 265 304.0 265 69 33.67171844429012
*A tsa04 0 265 304.0 265 90 576.4249130593298
*A tsa04 0 265 304.0 265 110 1761.4523214776045
*A tsa04 0 265 304.0 265 170 80.64025117873746
*A tsa04 0 265 304.0 265 240 3089.4496359865766
*A tsa04 0 267 304.0 267 227 193.1906574805849
*A tsa04 0 267 304.0 267 240 4024.66819238762
*A tsa04 0 298 204.0 298 90 20.37926483400676
*A tsa04 0 298 204.0 298 130 78.2152554853631
*A tsa04 0 298 204.0 298 150 169.10690299915592
*A tsa04 0 298 204.0 298 170 366.99435032460514
*A tsa04 0 298 204.0 298 240 671.8094816826357
*A tsa04 0 306 100.0 306 170 62.96505307039369
*A tsa04 0 306 100.0 306 240 1525.820038788041
*A tsa04 0 505 1200.0 505 150 58.91978498416027
*A tsa04 0 509 1211.0 509 110 132.99981336410383
*A tsa04 0 514 301.0 514 240 460.9101078262238
*A tsa04 0 514 301.0 514 242 112.03730757251444
*A tsa04 0 514 301.0 514 342 1104.0320608069642
*A tsa04 0 515 301.0 515 342 10.54595970889039
*A tsa04 0 536 403.0 536 342 532.3584069831406
*A tsa04 0 546 204.0 546 240 38.44859877263572
*A tsa04 0 548 100.0 548 342 100.58307908595766
*A tsa04 0 689 1211.0 689 90 296.9621232292924
*A tsa04 0 689 1211.0 689 110 17.893338285218285
*A tsa04 0 698 304.0 698 240 461.58240256060424
*A tsa04 0 728 204.0 728 150 194.09153165770772
*A tsa04 0 728 204.0 728 240 31.075706301696975
*A tsa04 0 733 100.0 733 150 8.42809326788804
*A tsa04 0 733 100.0 733 170 134.96473503843217
*A tsa04 0 733 100.0 733 240 476.3147378496505
*A tsa04 0 756 1211.0 756 70 5.203510177832139
*A tsa04 0 762 304.0 762 240 821.567396193198
*A tsa04 0 767 204.0 767 64 33.07168557908729
*A tsa04 0 767 204.0 767 150 89.26178672013667
*A tsa04 0 1026 100.0 1026 240 12.859318098811473
*A tsa04 0 1085 100.0 1085 240 220.8433745178325
*A tsa04 0 1191 204.0 1191 170 248.78025918727133
*A tsa04 0 1203 100.0 1203 240 38.475524444714765
"""

In [ ]:
with open('data/woodstock_model_files/tsa04.are', 'w') as f: f.write(areas_section)

# `YIELDS` section

In [ ]:
fm.yields

In [ ]:
# Reading and registering curve points
for AU, au_row in curve_points_table.iterrows():
    yname = 's%04d' % int(au_row.canfi_species)    
    curve_id = au_row.curve_id
    mask = ('?', '?', str(AU), '?', str(curve_id))
    points = [(r.x, r.y) for _, r in points_table.loc[AU].iterrows() if not r.x % period_length and r.x <= max_age]    
    c = fm.register_curve(ws3.core.Curve(yname, points=points, type='a', is_volume=True, xmax=fm.max_age, period_length=period_length))
    print('*Y', ' '.join(v for v in mask))
    print(yname, '1', ' '.join(str(int(c[x])) for x in range(0, 350, 10)))

In [ ]:
yields_section = \
"""
*Y ? ? 170 ? 170
s0301 1 0 0 0 0 1 11 40 89 151 218 284 349 409 464 514 514 589 611 627 638 647 651 652 653 654 656 657 658 659 660 662 662 662 662 662
*Y ? ? 201 ? 201
s0402 1 0 0 0 0 5 24 57 100 149 201 254 304 352 396 437 437 498 516 530 539 546 549 550 551 552 553 553 553 553 553 553 553 553 553 553
*Y ? ? 514 ? 514
s0301 1 0 0 0 0 0 3 20 56 110 167 224 280 332 381 425 425 495 516 531 542 549 554 556 557 558 559 560 560 561 562 563 564 565 565 565
*Y ? ? 536 ? 536
s0403 1 0 0 0 0 0 0 2 6 17 36 67 106 151 202 256 256 345 373 394 408 419 422 422 422 423 423 423 423 424 424 425 425 425 425 425
*Y ? ? 548 ? 548
s0100 1 0 0 0 0 2 9 19 41 72 111 154 197 240 282 321 321 376 391 402 408 412 412 410 409 408 406 405 403 402 401 399 398 397 397 397
*Y ? ? 515 ? 515
s0301 1 0 0 0 1 17 78 182 312 441 557 661 754 836 908 967 967 1051 1072 1085 1095 1102 1107 1108 1109 1111 1112 1113 1114 1116 1117 1118 1120 1121 1121 1121
*Y ? ? 505 ? 505
s1200 1 0 0 1 18 49 92 134 187 234 276 315 349 378 403 426 426 457 465 471 475 479 480 480 480 480 480 480 480 480 480 480 480 480 480 480
*Y ? ? 509 ? 509
s1211 1 0 0 0 0 2 6 14 22 31 41 51 61 70 78 84 84 92 95 97 98 99 99 99 99 99 99 99 99 99 99 99 99 99 99 99
*Y ? ? 509 ? 509
s1205 1 0 0 0 0 2 6 14 22 31 41 51 61 70 78 84 84 92 95 97 98 99 99 99 99 99 99 99 99 99 99 99 99 99 99 99
*Y ? ? 546 ? 546
s0204 1 0 0 0 0 0 8 23 46 71 96 120 144 166 185 202 202 223 230 234 236 238 238 238 237 237 237 236 236 235 235 234 234 234 234 234
*Y ? ? 265 ? 265
s0304 1 0 0 0 0 0 0 3 10 20 35 53 74 98 122 147 147 188 201 210 215 219 221 221 221 221 221 221 221 221 221 221 221 221 221 221
*Y ? ? 1085 ? 1085
s0100 1 0 0 0 0 0 0 0 0 0 0 1 1 1 2 6 6 20 28 35 41 45 48 48 48 48 52 53 53 53 53 55 58 58 58 58
*Y ? ? 306 ? 306
s0100 1 0 0 0 0 0 0 3 11 25 46 71 100 130 161 192 192 243 259 269 276 280 282 281 280 279 278 278 277 276 275 275 274 273 273 273
*Y ? ? 267 ? 267
s0304 1 0 0 0 27 94 190 289 374 445 499 544 582 612 636 654 654 676 681 685 688 691 693 695 696 697 699 700 701 703 704 706 707 706 705 704
*Y ? ? 256 ? 256
s1211 1 0 0 0 0 2 9 24 43 67 92 116 138 159 177 193 193 218 226 233 238 241 244 245 245 246 247 248 248 249 250 251 252 252 252 252
*Y ? ? 298 ? 298
s0204 1 0 0 0 0 1 8 23 42 65 88 110 130 148 164 180 180 204 213 219 224 227 229 230 231 231 232 232 233 233 234 235 235 236 236 236
*Y ? ? 698 ? 698
s0304 1 0 0 0 0 0 1 7 22 44 73 104 137 169 200 227 227 269 281 287 291 293 294 293 292 291 291 290 289 289 288 287 287 286 286 286
*Y ? ? 733 ? 733
s0100 1 0 0 0 0 0 1 6 19 39 66 97 130 164 196 226 226 274 287 295 299 301 300 298 296 294 293 292 290 289 287 286 284 283 282 282
*Y ? ? 54 ? 54
s1211 1 0 0 0 0 2 11 31 57 88 118 148 175 199 220 237 237 261 267 271 273 274 275 273 272 271 270 269 267 266 265 264 263 262 262 262
*Y ? ? 60 ? 60
s0304 1 0 0 0 0 0 1 8 21 41 66 95 126 156 186 212 212 250 260 266 269 270 270 269 268 267 266 265 264 264 263 262 261 260 260 260
*Y ? ? 767 ? 767
s0204 1 0 0 0 0 0 3 12 26 44 63 81 99 115 129 141 141 157 161 164 165 166 166 166 165 165 165 164 164 163 163 162 162 161 161 161
*Y ? ? 78 ? 78
s0100 1 0 0 0 0 0 0 1 7 18 33 53 76 100 124 149 149 189 201 209 213 216 215 213 211 209 208 206 205 203 202 200 199 197 197 197
*Y ? ? 762 ? 762
s0304 1 0 0 0 0 5 35 85 145 207 265 318 363 401 430 450 450 467 468 467 466 465 463 462 461 460 458 457 456 454 453 452 450 450 450 450
*Y ? ? 74 ? 74
s0204 1 0 0 0 0 0 6 22 44 70 95 120 142 161 178 192 192 210 214 217 218 218 218 216 215 214 213 212 210 209 208 207 206 205 205 205
*Y ? ? 1203 ? 1203
s0100 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 4 7 10 12 14 15 15 15 15 15 15 15 15 15 15 16 16 16 16
*Y ? ? 728 ? 728
s0204 1 0 0 0 2 19 59 110 161 205 244 276 302 325 344 360 360 382 388 392 395 397 398 397 397 396 396 395 395 394 394 393 393 393 393 393
*Y ? ? 1191 ? 1191
s0204 1 0 0 0 0 0 0 0 1 2 4 5 7 9 12 16 16 23 25 27 29 30 30 30 31 31 31 31 31 31 31 32 32 32 32 32
*Y ? ? 124 ? 124
s0302 1 0 0 0 0 0 0 4 13 33 56 83 111 140 168 195 195 239 254 264 272 278 282 283 284 285 286 287 288 289 290 291 292 294 294 294
*Y ? ? 689 ? 689
s1211 1 0 0 0 0 5 18 40 66 96 126 155 182 206 226 244 244 269 277 282 286 289 291 292 292 292 293 293 294 294 294 295 295 295 295 295
*Y ? ? 61 ? 61
s0304 1 0 0 0 1 9 39 90 151 215 276 330 376 413 440 461 461 481 482 482 481 480 478 477 475 473 472 470 469 467 465 464 462 461 461 461
*Y ? ? 81 ? 81
s0101 1 0 0 0 0 0 0 1 4 10 19 31 46 62 79 96 96 128 139 146 150 152 154 154 153 153 152 152 151 151 150 150 149 149 148 148
*Y ? ? 756 ? 756
s1211 1 0 0 0 0 0 5 16 34 56 79 102 125 145 164 179 179 200 205 208 211 212 213 213 212 212 211 211 210 210 209 209 208 208 208 208
*Y ? ? 79 ? 79
s0100 1 0 0 0 0 0 6 30 74 130 189 245 294 336 373 403 403 442 450 452 452 450 447 441 436 431 426 422 417 414 410 406 402 399 397 397
*Y ? ? 1026 ? 1026
s0100 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 3 4 5 6 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7
*YC ? ? ? ? ?
totvol _SUM(s101.0, s402.0, s1200.0, s304.0, s100.0, s204.0, s403.0, s301.0, s302.0, s1211.0)
"""

In [ ]:
with open('data/woodstock_model_files/tsa04.yld', 'w') as f: f.write(yields_section)

# `ACTIONS` section

In [ ]:
actions_section = \
"""
ACTIONS
*ACTION harvest 0
*OPERABLE harvest
? ? ? ? ? _AGE >= 100 AND _AGE <= 600
"""

In [ ]:
with open('data/woodstock_model_files/tsa04.act', 'w') as f: f.write(actions_section)

# `TRANSITIONS` section

In [ ]:
# Setting up an action
acode = 'harvest'
print('*CASE', acode)
for au_id, au_row in stands_table.iterrows():
    if not au_row.theme1: continue
    target_curve_id = au_row.theme4  
    smask = ' '.join(('?', '?' , str (target_curve_id), '?', '?'))
    tmask = ' '.join(('?', '?' , '?', '?', str(target_curve_id)))
    print('*SOURCE', smask)
    print('*TARGET', tmask, '1.0')

In [ ]:
transitions_section = \
"""
*CASE harvest
*SOURCE ? ? 265 ? ?
*TARGET ? ? ? ? 265 1.0
*SOURCE ? ? 265 ? ?
*TARGET ? ? ? ? 265 1.0
*SOURCE ? ? 265 ? ?
*TARGET ? ? ? ? 265 1.0
*SOURCE ? ? 387 ? ?
*TARGET ? ? ? ? 387 1.0
*SOURCE ? ? 265 ? ?
*TARGET ? ? ? ? 265 1.0
*SOURCE ? ? 335 ? ?
*TARGET ? ? ? ? 335 1.0
*SOURCE ? ? 387 ? ?
*TARGET ? ? ? ? 387 1.0
*SOURCE ? ? 344 ? ?
*TARGET ? ? ? ? 344 1.0
*SOURCE ? ? 306 ? ?
*TARGET ? ? ? ? 306 1.0
*SOURCE ? ? 365 ? ?
*TARGET ? ? ? ? 365 1.0
*SOURCE ? ? 387 ? ?
*TARGET ? ? ? ? 387 1.0
*SOURCE ? ? 387 ? ?
*TARGET ? ? ? ? 387 1.0
*SOURCE ? ? 306 ? ?
*TARGET ? ? ? ? 306 1.0
*SOURCE ? ? 306 ? ?
*TARGET ? ? ? ? 306 1.0
*SOURCE ? ? 388 ? ?
*TARGET ? ? ? ? 388 1.0
*SOURCE ? ? 388 ? ?
*TARGET ? ? ? ? 388 1.0
*SOURCE ? ? 387 ? ?
*TARGET ? ? ? ? 387 1.0
*SOURCE ? ? 265 ? ?
*TARGET ? ? ? ? 265 1.0
*SOURCE ? ? 387 ? ?
*TARGET ? ? ? ? 387 1.0
*SOURCE ? ? 306 ? ?
*TARGET ? ? ? ? 306 1.0
*SOURCE ? ? 265 ? ?
*TARGET ? ? ? ? 265 1.0
*SOURCE ? ? 265 ? ?
*TARGET ? ? ? ? 265 1.0
*SOURCE ? ? 387 ? ?
*TARGET ? ? ? ? 387 1.0
*SOURCE ? ? 265 ? ?
*TARGET ? ? ? ? 265 1.0
*SOURCE ? ? 306 ? ?
*TARGET ? ? ? ? 306 1.0
*SOURCE ? ? 265 ? ?
*TARGET ? ? ? ? 265 1.0
*SOURCE ? ? 265 ? ?
*TARGET ? ? ? ? 265 1.0
*SOURCE ? ? 388 ? ?
*TARGET ? ? ? ? 388 1.0
*SOURCE ? ? 265 ? ?
*TARGET ? ? ? ? 265 1.0
*SOURCE ? ? 387 ? ?
*TARGET ? ? ? ? 387 1.0
*SOURCE ? ? 388 ? ?
*TARGET ? ? ? ? 388 1.0
*SOURCE ? ? 306 ? ?
*TARGET ? ? ? ? 306 1.0
*SOURCE ? ? 365 ? ?
*TARGET ? ? ? ? 365 1.0
*SOURCE ? ? 265 ? ?
*TARGET ? ? ? ? 265 1.0
*SOURCE ? ? 400 ? ?
*TARGET ? ? ? ? 400 1.0
*SOURCE ? ? 387 ? ?
*TARGET ? ? ? ? 387 1.0
*SOURCE ? ? 265 ? ?
*TARGET ? ? ? ? 265 1.0
*SOURCE ? ? 306 ? ?
*TARGET ? ? ? ? 306 1.0
*SOURCE ? ? 387 ? ?
*TARGET ? ? ? ? 387 1.0
*SOURCE ? ? 365 ? ?
*TARGET ? ? ? ? 365 1.0
*SOURCE ? ? 388 ? ?
*TARGET ? ? ? ? 388 1.0
*SOURCE ? ? 387 ? ?
*TARGET ? ? ? ? 387 1.0
*SOURCE ? ? 387 ? ?
*TARGET ? ? ? ? 387 1.0
*SOURCE ? ? 388 ? ?
*TARGET ? ? ? ? 388 1.0
*SOURCE ? ? 388 ? ?
*TARGET ? ? ? ? 388 1.0
*SOURCE ? ? 306 ? ?
*TARGET ? ? ? ? 306 1.0
*SOURCE ? ? 388 ? ?
*TARGET ? ? ? ? 388 1.0
*SOURCE ? ? 387 ? ?
*TARGET ? ? ? ? 387 1.0
*SOURCE ? ? 265 ? ?
*TARGET ? ? ? ? 265 1.0
*SOURCE ? ? 307 ? ?
*TARGET ? ? ? ? 307 1.0
*SOURCE ? ? 387 ? ?
*TARGET ? ? ? ? 387 1.0
*SOURCE ? ? 365 ? ?
*TARGET ? ? ? ? 365 1.0
*SOURCE ? ? 387 ? ?
*TARGET ? ? ? ? 387 1.0
*SOURCE ? ? 387 ? ?
*TARGET ? ? ? ? 387 1.0
*SOURCE ? ? 387 ? ?
*TARGET ? ? ? ? 387 1.0
*SOURCE ? ? 387 ? ?
*TARGET ? ? ? ? 387 1.0
*SOURCE ? ? 265 ? ?
*TARGET ? ? ? ? 265 1.0
*SOURCE ? ? 387 ? ?
*TARGET ? ? ? ? 387 1.0
*SOURCE ? ? 388 ? ?
*TARGET ? ? ? ? 388 1.0
*SOURCE ? ? 365 ? ?
*TARGET ? ? ? ? 365 1.0
*SOURCE ? ? 306 ? ?
*TARGET ? ? ? ? 306 1.0
*SOURCE ? ? 307 ? ?
*TARGET ? ? ? ? 307 1.0
*SOURCE ? ? 306 ? ?
*TARGET ? ? ? ? 306 1.0
*SOURCE ? ? 387 ? ?
*TARGET ? ? ? ? 387 1.0
*SOURCE ? ? 265 ? ?
*TARGET ? ? ? ? 265 1.0
*SOURCE ? ? 344 ? ?
*TARGET ? ? ? ? 344 1.0
*SOURCE ? ? 365 ? ?
*TARGET ? ? ? ? 365 1.0
*SOURCE ? ? 265 ? ?
*TARGET ? ? ? ? 265 1.0
*SOURCE ? ? 387 ? ?
*TARGET ? ? ? ? 387 1.0
*SOURCE ? ? 306 ? ?
*TARGET ? ? ? ? 306 1.0
*SOURCE ? ? 265 ? ?
*TARGET ? ? ? ? 265 1.0
*SOURCE ? ? 387 ? ?
*TARGET ? ? ? ? 387 1.0
*SOURCE ? ? 388 ? ?
*TARGET ? ? ? ? 388 1.0
*SOURCE ? ? 387 ? ?
*TARGET ? ? ? ? 387 1.0
*SOURCE ? ? 388 ? ?
*TARGET ? ? ? ? 388 1.0
*SOURCE ? ? 265 ? ?
*TARGET ? ? ? ? 265 1.0
*SOURCE ? ? 365 ? ?
*TARGET ? ? ? ? 365 1.0
*SOURCE ? ? 335 ? ?
*TARGET ? ? ? ? 335 1.0
*SOURCE ? ? 387 ? ?
*TARGET ? ? ? ? 387 1.0
*SOURCE ? ? 388 ? ?
*TARGET ? ? ? ? 388 1.0
*SOURCE ? ? 265 ? ?
*TARGET ? ? ? ? 265 1.0
*SOURCE ? ? 265 ? ?
*TARGET ? ? ? ? 265 1.0
*SOURCE ? ? 387 ? ?
*TARGET ? ? ? ? 387 1.0
*SOURCE ? ? 399 ? ?
*TARGET ? ? ? ? 399 1.0
*SOURCE ? ? 265 ? ?
*TARGET ? ? ? ? 265 1.0
*SOURCE ? ? 387 ? ?
*TARGET ? ? ? ? 387 1.0
*SOURCE ? ? 387 ? ?
*TARGET ? ? ? ? 387 1.0
*SOURCE ? ? 399 ? ?
*TARGET ? ? ? ? 399 1.0
*SOURCE ? ? 387 ? ?
*TARGET ? ? ? ? 387 1.0
*SOURCE ? ? 387 ? ?
*TARGET ? ? ? ? 387 1.0
*SOURCE ? ? 365 ? ?
*TARGET ? ? ? ? 365 1.0
*SOURCE ? ? 400 ? ?
*TARGET ? ? ? ? 400 1.0
*SOURCE ? ? 387 ? ?
*TARGET ? ? ? ? 387 1.0
*SOURCE ? ? 265 ? ?
*TARGET ? ? ? ? 265 1.0
*SOURCE ? ? 387 ? ?
*TARGET ? ? ? ? 387 1.0
*SOURCE ? ? 387 ? ?
*TARGET ? ? ? ? 387 1.0
*SOURCE ? ? 387 ? ?
*TARGET ? ? ? ? 387 1.0
*SOURCE ? ? 265 ? ?
*TARGET ? ? ? ? 265 1.0
*SOURCE ? ? 306 ? ?
*TARGET ? ? ? ? 306 1.0
*SOURCE ? ? 387 ? ?
*TARGET ? ? ? ? 387 1.0
*SOURCE ? ? 265 ? ?
*TARGET ? ? ? ? 265 1.0
*SOURCE ? ? 344 ? ?
*TARGET ? ? ? ? 344 1.0
*SOURCE ? ? 388 ? ?
*TARGET ? ? ? ? 388 1.0
*SOURCE ? ? 306 ? ?
*TARGET ? ? ? ? 306 1.0
*SOURCE ? ? 306 ? ?
*TARGET ? ? ? ? 306 1.0
*SOURCE ? ? 265 ? ?
*TARGET ? ? ? ? 265 1.0
*SOURCE ? ? 388 ? ?
*TARGET ? ? ? ? 388 1.0
*SOURCE ? ? 317 ? ?
*TARGET ? ? ? ? 317 1.0
*SOURCE ? ? 387 ? ?
*TARGET ? ? ? ? 387 1.0
*SOURCE ? ? 265 ? ?
*TARGET ? ? ? ? 265 1.0
*SOURCE ? ? 387 ? ?
*TARGET ? ? ? ? 387 1.0
*SOURCE ? ? 344 ? ?
*TARGET ? ? ? ? 344 1.0
*SOURCE ? ? 365 ? ?
*TARGET ? ? ? ? 365 1.0
*SOURCE ? ? 306 ? ?
*TARGET ? ? ? ? 306 1.0
*SOURCE ? ? 387 ? ?
*TARGET ? ? ? ? 387 1.0
*SOURCE ? ? 387 ? ?
*TARGET ? ? ? ? 387 1.0
*SOURCE ? ? 306 ? ?
*TARGET ? ? ? ? 306 1.0
*SOURCE ? ? 387 ? ?
*TARGET ? ? ? ? 387 1.0
*SOURCE ? ? 400 ? ?
*TARGET ? ? ? ? 400 1.0
*SOURCE ? ? 335 ? ?
*TARGET ? ? ? ? 335 1.0
*SOURCE ? ? 388 ? ?
*TARGET ? ? ? ? 388 1.0
*SOURCE ? ? 265 ? ?
*TARGET ? ? ? ? 265 1.0
*SOURCE ? ? 306 ? ?
*TARGET ? ? ? ? 306 1.0
*SOURCE ? ? 387 ? ?
*TARGET ? ? ? ? 387 1.0
*SOURCE ? ? 306 ? ?
*TARGET ? ? ? ? 306 1.0
*SOURCE ? ? 306 ? ?
*TARGET ? ? ? ? 306 1.0
*SOURCE ? ? 306 ? ?
*TARGET ? ? ? ? 306 1.0
*SOURCE ? ? 265 ? ?
*TARGET ? ? ? ? 265 1.0
*SOURCE ? ? 306 ? ?
*TARGET ? ? ? ? 306 1.0
*SOURCE ? ? 399 ? ?
*TARGET ? ? ? ? 399 1.0
*SOURCE ? ? 365 ? ?
*TARGET ? ? ? ? 365 1.0
*SOURCE ? ? 365 ? ?
*TARGET ? ? ? ? 365 1.0
*SOURCE ? ? 317 ? ?
*TARGET ? ? ? ? 317 1.0
*SOURCE ? ? 265 ? ?
*TARGET ? ? ? ? 265 1.0
*SOURCE ? ? 265 ? ?
*TARGET ? ? ? ? 265 1.0
*SOURCE ? ? 387 ? ?
*TARGET ? ? ? ? 387 1.0
*SOURCE ? ? 387 ? ?
*TARGET ? ? ? ? 387 1.0
*SOURCE ? ? 387 ? ?
*TARGET ? ? ? ? 387 1.0
*SOURCE ? ? 388 ? ?
*TARGET ? ? ? ? 388 1.0
*SOURCE ? ? 306 ? ?
*TARGET ? ? ? ? 306 1.0
*SOURCE ? ? 365 ? ?
*TARGET ? ? ? ? 365 1.0
*SOURCE ? ? 265 ? ?
*TARGET ? ? ? ? 265 1.0
*SOURCE ? ? 306 ? ?
*TARGET ? ? ? ? 306 1.0
*SOURCE ? ? 306 ? ?
*TARGET ? ? ? ? 306 1.0
*SOURCE ? ? 387 ? ?
*TARGET ? ? ? ? 387 1.0
*SOURCE ? ? 265 ? ?
*TARGET ? ? ? ? 265 1.0
*SOURCE ? ? 387 ? ?
*TARGET ? ? ? ? 387 1.0
*SOURCE ? ? 265 ? ?
*TARGET ? ? ? ? 265 1.0
*SOURCE ? ? 388 ? ?
*TARGET ? ? ? ? 388 1.0
*SOURCE ? ? 307 ? ?
*TARGET ? ? ? ? 307 1.0
*SOURCE ? ? 265 ? ?
*TARGET ? ? ? ? 265 1.0
*SOURCE ? ? 387 ? ?
*TARGET ? ? ? ? 387 1.0
*SOURCE ? ? 388 ? ?
*TARGET ? ? ? ? 388 1.0
*SOURCE ? ? 388 ? ?
*TARGET ? ? ? ? 388 1.0
*SOURCE ? ? 306 ? ?
*TARGET ? ? ? ? 306 1.0
*SOURCE ? ? 388 ? ?
*TARGET ? ? ? ? 388 1.0
*SOURCE ? ? 388 ? ?
*TARGET ? ? ? ? 388 1.0
*SOURCE ? ? 265 ? ?
*TARGET ? ? ? ? 265 1.0
*SOURCE ? ? 265 ? ?
*TARGET ? ? ? ? 265 1.0
*SOURCE ? ? 306 ? ?
*TARGET ? ? ? ? 306 1.0
*SOURCE ? ? 265 ? ?
*TARGET ? ? ? ? 265 1.0
*SOURCE ? ? 387 ? ?
*TARGET ? ? ? ? 387 1.0
*SOURCE ? ? 265 ? ?
*TARGET ? ? ? ? 265 1.0
*SOURCE ? ? 366 ? ?
*TARGET ? ? ? ? 366 1.0
*SOURCE ? ? 387 ? ?
*TARGET ? ? ? ? 387 1.0
*SOURCE ? ? 365 ? ?
*TARGET ? ? ? ? 365 1.0
*SOURCE ? ? 365 ? ?
*TARGET ? ? ? ? 365 1.0
*SOURCE ? ? 306 ? ?
*TARGET ? ? ? ? 306 1.0
*SOURCE ? ? 387 ? ?
*TARGET ? ? ? ? 387 1.0
*SOURCE ? ? 387 ? ?
*TARGET ? ? ? ? 387 1.0
*SOURCE ? ? 306 ? ?
*TARGET ? ? ? ? 306 1.0
*SOURCE ? ? 388 ? ?
*TARGET ? ? ? ? 388 1.0
*SOURCE ? ? 344 ? ?
*TARGET ? ? ? ? 344 1.0
*SOURCE ? ? 387 ? ?
*TARGET ? ? ? ? 387 1.0
*SOURCE ? ? 387 ? ?
*TARGET ? ? ? ? 387 1.0
*SOURCE ? ? 265 ? ?
*TARGET ? ? ? ? 265 1.0
*SOURCE ? ? 265 ? ?
*TARGET ? ? ? ? 265 1.0
*SOURCE ? ? 265 ? ?
*TARGET ? ? ? ? 265 1.0
*SOURCE ? ? 387 ? ?
*TARGET ? ? ? ? 387 1.0
*SOURCE ? ? 344 ? ?
*TARGET ? ? ? ? 344 1.0
*SOURCE ? ? 347 ? ?
*TARGET ? ? ? ? 347 1.0
*SOURCE ? ? 365 ? ?
*TARGET ? ? ? ? 365 1.0
*SOURCE ? ? 365 ? ?
*TARGET ? ? ? ? 365 1.0
*SOURCE ? ? 387 ? ?
*TARGET ? ? ? ? 387 1.0
*SOURCE ? ? 388 ? ?
*TARGET ? ? ? ? 388 1.0
*SOURCE ? ? 265 ? ?
*TARGET ? ? ? ? 265 1.0
*SOURCE ? ? 365 ? ?
*TARGET ? ? ? ? 365 1.0
*SOURCE ? ? 365 ? ?
*TARGET ? ? ? ? 365 1.0
*SOURCE ? ? 400 ? ?
*TARGET ? ? ? ? 400 1.0
*SOURCE ? ? 306 ? ?
*TARGET ? ? ? ? 306 1.0
*SOURCE ? ? 388 ? ?
*TARGET ? ? ? ? 388 1.0
*SOURCE ? ? 388 ? ?
*TARGET ? ? ? ? 388 1.0
*SOURCE ? ? 387 ? ?
*TARGET ? ? ? ? 387 1.0
*SOURCE ? ? 265 ? ?
*TARGET ? ? ? ? 265 1.0
*SOURCE ? ? 345 ? ?
*TARGET ? ? ? ? 345 1.0
*SOURCE ? ? 365 ? ?
*TARGET ? ? ? ? 365 1.0
*SOURCE ? ? 387 ? ?
*TARGET ? ? ? ? 387 1.0
*SOURCE ? ? 265 ? ?
*TARGET ? ? ? ? 265 1.0
*SOURCE ? ? 306 ? ?
*TARGET ? ? ? ? 306 1.0
*SOURCE ? ? 265 ? ?
*TARGET ? ? ? ? 265 1.0
*SOURCE ? ? 388 ? ?
*TARGET ? ? ? ? 388 1.0
*SOURCE ? ? 307 ? ?
*TARGET ? ? ? ? 307 1.0
*SOURCE ? ? 387 ? ?
*TARGET ? ? ? ? 387 1.0
*SOURCE ? ? 388 ? ?
*TARGET ? ? ? ? 388 1.0
*SOURCE ? ? 388 ? ?
*TARGET ? ? ? ? 388 1.0
*SOURCE ? ? 388 ? ?
*TARGET ? ? ? ? 388 1.0
*SOURCE ? ? 387 ? ?
*TARGET ? ? ? ? 387 1.0
*SOURCE ? ? 388 ? ?
*TARGET ? ? ? ? 388 1.0
*SOURCE ? ? 387 ? ?
*TARGET ? ? ? ? 387 1.0
*SOURCE ? ? 306 ? ?
*TARGET ? ? ? ? 306 1.0
*SOURCE ? ? 306 ? ?
*TARGET ? ? ? ? 306 1.0
*SOURCE ? ? 387 ? ?
*TARGET ? ? ? ? 387 1.0
*SOURCE ? ? 344 ? ?
*TARGET ? ? ? ? 344 1.0
*SOURCE ? ? 265 ? ?
*TARGET ? ? ? ? 265 1.0
*SOURCE ? ? 306 ? ?
*TARGET ? ? ? ? 306 1.0
*SOURCE ? ? 347 ? ?
*TARGET ? ? ? ? 347 1.0
*SOURCE ? ? 365 ? ?
*TARGET ? ? ? ? 365 1.0
*SOURCE ? ? 265 ? ?
*TARGET ? ? ? ? 265 1.0
*SOURCE ? ? 265 ? ?
*TARGET ? ? ? ? 265 1.0
*SOURCE ? ? 365 ? ?
*TARGET ? ? ? ? 365 1.0
*SOURCE ? ? 388 ? ?
*TARGET ? ? ? ? 388 1.0
*SOURCE ? ? 388 ? ?
*TARGET ? ? ? ? 388 1.0
*SOURCE ? ? 265 ? ?
*TARGET ? ? ? ? 265 1.0
*SOURCE ? ? 344 ? ?
*TARGET ? ? ? ? 344 1.0
*SOURCE ? ? 257 ? ?
*TARGET ? ? ? ? 257 1.0
*SOURCE ? ? 387 ? ?
*TARGET ? ? ? ? 387 1.0
*SOURCE ? ? 344 ? ?
*TARGET ? ? ? ? 344 1.0
*SOURCE ? ? 265 ? ?
*TARGET ? ? ? ? 265 1.0
*SOURCE ? ? 388 ? ?
*TARGET ? ? ? ? 388 1.0
*SOURCE ? ? 387 ? ?
*TARGET ? ? ? ? 387 1.0
*SOURCE ? ? 344 ? ?
*TARGET ? ? ? ? 344 1.0
*SOURCE ? ? 265 ? ?
*TARGET ? ? ? ? 265 1.0
*SOURCE ? ? 387 ? ?
*TARGET ? ? ? ? 387 1.0
*SOURCE ? ? 388 ? ?
*TARGET ? ? ? ? 388 1.0
*SOURCE ? ? 387 ? ?
*TARGET ? ? ? ? 387 1.0
*SOURCE ? ? 387 ? ?
*TARGET ? ? ? ? 387 1.0
*SOURCE ? ? 265 ? ?
*TARGET ? ? ? ? 265 1.0
*SOURCE ? ? 265 ? ?
*TARGET ? ? ? ? 265 1.0
*SOURCE ? ? 365 ? ?
*TARGET ? ? ? ? 365 1.0
*SOURCE ? ? 388 ? ?
*TARGET ? ? ? ? 388 1.0
*SOURCE ? ? 388 ? ?
*TARGET ? ? ? ? 388 1.0
*SOURCE ? ? 365 ? ?
*TARGET ? ? ? ? 365 1.0
*SOURCE ? ? 306 ? ?
*TARGET ? ? ? ? 306 1.0
*SOURCE ? ? 387 ? ?
*TARGET ? ? ? ? 387 1.0
*SOURCE ? ? 387 ? ?
*TARGET ? ? ? ? 387 1.0
*SOURCE ? ? 387 ? ?
*TARGET ? ? ? ? 387 1.0
*SOURCE ? ? 265 ? ?
*TARGET ? ? ? ? 265 1.0
*SOURCE ? ? 388 ? ?
*TARGET ? ? ? ? 388 1.0
*SOURCE ? ? 265 ? ?
*TARGET ? ? ? ? 265 1.0
*SOURCE ? ? 344 ? ?
*TARGET ? ? ? ? 344 1.0
*SOURCE ? ? 733 ? ?
*TARGET ? ? ? ? 733 1.0
*SOURCE ? ? 388 ? ?
*TARGET ? ? ? ? 388 1.0
*SOURCE ? ? 387 ? ?
*TARGET ? ? ? ? 387 1.0
*SOURCE ? ? 296 ? ?
*TARGET ? ? ? ? 296 1.0
*SOURCE ? ? 365 ? ?
*TARGET ? ? ? ? 365 1.0
*SOURCE ? ? 356 ? ?
*TARGET ? ? ? ? 356 1.0
*SOURCE ? ? 378 ? ?
*TARGET ? ? ? ? 378 1.0
*SOURCE ? ? 387 ? ?
*TARGET ? ? ? ? 387 1.0
*SOURCE ? ? 265 ? ?
*TARGET ? ? ? ? 265 1.0
*SOURCE ? ? 387 ? ?
*TARGET ? ? ? ? 387 1.0
*SOURCE ? ? 265 ? ?
*TARGET ? ? ? ? 265 1.0
*SOURCE ? ? 733 ? ?
*TARGET ? ? ? ? 733 1.0
*SOURCE ? ? 387 ? ?
*TARGET ? ? ? ? 387 1.0
*SOURCE ? ? 387 ? ?
*TARGET ? ? ? ? 387 1.0
*SOURCE ? ? 387 ? ?
*TARGET ? ? ? ? 387 1.0
*SOURCE ? ? 387 ? ?
*TARGET ? ? ? ? 387 1.0
*SOURCE ? ? 356 ? ?
*TARGET ? ? ? ? 356 1.0
*SOURCE ? ? 387 ? ?
*TARGET ? ? ? ? 387 1.0
*SOURCE ? ? 365 ? ?
*TARGET ? ? ? ? 365 1.0
*SOURCE ? ? 387 ? ?
*TARGET ? ? ? ? 387 1.0
*SOURCE ? ? 366 ? ?
*TARGET ? ? ? ? 366 1.0
*SOURCE ? ? 365 ? ?
*TARGET ? ? ? ? 365 1.0
*SOURCE ? ? 702 ? ?
*TARGET ? ? ? ? 702 1.0
*SOURCE ? ? 357 ? ?
*TARGET ? ? ? ? 357 1.0
*SOURCE ? ? 711 ? ?
*TARGET ? ? ? ? 711 1.0
*SOURCE ? ? 733 ? ?
*TARGET ? ? ? ? 733 1.0
*SOURCE ? ? 356 ? ?
*TARGET ? ? ? ? 356 1.0
*SOURCE ? ? 347 ? ?
*TARGET ? ? ? ? 347 1.0
*SOURCE ? ? 356 ? ?
*TARGET ? ? ? ? 356 1.0
*SOURCE ? ? 734 ? ?
*TARGET ? ? ? ? 734 1.0
*SOURCE ? ? 387 ? ?
*TARGET ? ? ? ? 387 1.0
*SOURCE ? ? 387 ? ?
*TARGET ? ? ? ? 387 1.0
*SOURCE ? ? 365 ? ?
*TARGET ? ? ? ? 365 1.0
*SOURCE ? ? 387 ? ?
*TARGET ? ? ? ? 387 1.0
*SOURCE ? ? 365 ? ?
*TARGET ? ? ? ? 365 1.0
*SOURCE ? ? 387 ? ?
*TARGET ? ? ? ? 387 1.0
*SOURCE ? ? 387 ? ?
*TARGET ? ? ? ? 387 1.0
*SOURCE ? ? 379 ? ?
*TARGET ? ? ? ? 379 1.0
*SOURCE ? ? 387 ? ?
*TARGET ? ? ? ? 387 1.0
*SOURCE ? ? 711 ? ?
*TARGET ? ? ? ? 711 1.0
*SOURCE ? ? 734 ? ?
*TARGET ? ? ? ? 734 1.0
*SOURCE ? ? 265 ? ?
*TARGET ? ? ? ? 265 1.0
*SOURCE ? ? 265 ? ?
*TARGET ? ? ? ? 265 1.0
*SOURCE ? ? 365 ? ?
*TARGET ? ? ? ? 365 1.0
*SOURCE ? ? 734 ? ?
*TARGET ? ? ? ? 734 1.0
*SOURCE ? ? 387 ? ?
*TARGET ? ? ? ? 387 1.0
*SOURCE ? ? 265 ? ?
*TARGET ? ? ? ? 265 1.0
*SOURCE ? ? 365 ? ?
*TARGET ? ? ? ? 365 1.0
*SOURCE ? ? 387 ? ?
*TARGET ? ? ? ? 387 1.0
*SOURCE ? ? 378 ? ?
*TARGET ? ? ? ? 378 1.0
*SOURCE ? ? 356 ? ?
*TARGET ? ? ? ? 356 1.0
*SOURCE ? ? 265 ? ?
*TARGET ? ? ? ? 265 1.0
*SOURCE ? ? 357 ? ?
*TARGET ? ? ? ? 357 1.0
*SOURCE ? ? 387 ? ?
*TARGET ? ? ? ? 387 1.0
*SOURCE ? ? 344 ? ?
*TARGET ? ? ? ? 344 1.0
*SOURCE ? ? 344 ? ?
*TARGET ? ? ? ? 344 1.0
*SOURCE ? ? 335 ? ?
*TARGET ? ? ? ? 335 1.0
*SOURCE ? ? 265 ? ?
*TARGET ? ? ? ? 265 1.0
*SOURCE ? ? 733 ? ?
*TARGET ? ? ? ? 733 1.0
*SOURCE ? ? 379 ? ?
*TARGET ? ? ? ? 379 1.0
*SOURCE ? ? 682 ? ?
*TARGET ? ? ? ? 682 1.0
*SOURCE ? ? 366 ? ?
*TARGET ? ? ? ? 366 1.0
*SOURCE ? ? 365 ? ?
*TARGET ? ? ? ? 365 1.0
*SOURCE ? ? 357 ? ?
*TARGET ? ? ? ? 357 1.0
*SOURCE ? ? 265 ? ?
*TARGET ? ? ? ? 265 1.0
*SOURCE ? ? 265 ? ?
*TARGET ? ? ? ? 265 1.0
*SOURCE ? ? 265 ? ?
*TARGET ? ? ? ? 265 1.0
*SOURCE ? ? 733 ? ?
*TARGET ? ? ? ? 733 1.0
*SOURCE ? ? 733 ? ?
*TARGET ? ? ? ? 733 1.0
*SOURCE ? ? 733 ? ?
*TARGET ? ? ? ? 733 1.0
*SOURCE ? ? 306 ? ?
*TARGET ? ? ? ? 306 1.0
*SOURCE ? ? 733 ? ?
*TARGET ? ? ? ? 733 1.0
*SOURCE ? ? 265 ? ?
*TARGET ? ? ? ? 265 1.0
*SOURCE ? ? 306 ? ?
*TARGET ? ? ? ? 306 1.0
*SOURCE ? ? 306 ? ?
*TARGET ? ? ? ? 306 1.0
*SOURCE ? ? 388 ? ?
*TARGET ? ? ? ? 388 1.0
*SOURCE ? ? 357 ? ?
*TARGET ? ? ? ? 357 1.0
*SOURCE ? ? 379 ? ?
*TARGET ? ? ? ? 379 1.0
*SOURCE ? ? 265 ? ?
*TARGET ? ? ? ? 265 1.0
*SOURCE ? ? 733 ? ?
*TARGET ? ? ? ? 733 1.0
*SOURCE ? ? 702 ? ?
*TARGET ? ? ? ? 702 1.0
*SOURCE ? ? 711 ? ?
*TARGET ? ? ? ? 711 1.0
*SOURCE ? ? 378 ? ?
*TARGET ? ? ? ? 378 1.0
*SOURCE ? ? 365 ? ?
*TARGET ? ? ? ? 365 1.0
*SOURCE ? ? 366 ? ?
*TARGET ? ? ? ? 366 1.0
*SOURCE ? ? 733 ? ?
*TARGET ? ? ? ? 733 1.0
*SOURCE ? ? 306 ? ?
*TARGET ? ? ? ? 306 1.0
*SOURCE ? ? 733 ? ?
*TARGET ? ? ? ? 733 1.0
*SOURCE ? ? 728 ? ?
*TARGET ? ? ? ? 728 1.0
*SOURCE ? ? 711 ? ?
*TARGET ? ? ? ? 711 1.0
*SOURCE ? ? 387 ? ?
*TARGET ? ? ? ? 387 1.0
*SOURCE ? ? 702 ? ?
*TARGET ? ? ? ? 702 1.0
*SOURCE ? ? 710 ? ?
*TARGET ? ? ? ? 710 1.0
*SOURCE ? ? 265 ? ?
*TARGET ? ? ? ? 265 1.0
*SOURCE ? ? 733 ? ?
*TARGET ? ? ? ? 733 1.0
*SOURCE ? ? 344 ? ?
*TARGET ? ? ? ? 344 1.0
*SOURCE ? ? 265 ? ?
*TARGET ? ? ? ? 265 1.0
*SOURCE ? ? 306 ? ?
*TARGET ? ? ? ? 306 1.0
*SOURCE ? ? 306 ? ?
*TARGET ? ? ? ? 306 1.0
*SOURCE ? ? 733 ? ?
*TARGET ? ? ? ? 733 1.0
*SOURCE ? ? 296 ? ?
*TARGET ? ? ? ? 296 1.0
*SOURCE ? ? 387 ? ?
*TARGET ? ? ? ? 387 1.0
*SOURCE ? ? 387 ? ?
*TARGET ? ? ? ? 387 1.0
*SOURCE ? ? 711 ? ?
*TARGET ? ? ? ? 711 1.0
*SOURCE ? ? 265 ? ?
*TARGET ? ? ? ? 265 1.0
*SOURCE ? ? 265 ? ?
*TARGET ? ? ? ? 265 1.0
*SOURCE ? ? 734 ? ?
*TARGET ? ? ? ? 734 1.0
*SOURCE ? ? 265 ? ?
*TARGET ? ? ? ? 265 1.0
*SOURCE ? ? 336 ? ?
*TARGET ? ? ? ? 336 1.0
*SOURCE ? ? 365 ? ?
*TARGET ? ? ? ? 365 1.0
*SOURCE ? ? 733 ? ?
*TARGET ? ? ? ? 733 1.0
*SOURCE ? ? 379 ? ?
*TARGET ? ? ? ? 379 1.0
*SOURCE ? ? 265 ? ?
*TARGET ? ? ? ? 265 1.0
*SOURCE ? ? 1192 ? ?
*TARGET ? ? ? ? 1192 1.0
*SOURCE ? ? 306 ? ?
*TARGET ? ? ? ? 306 1.0
*SOURCE ? ? 307 ? ?
*TARGET ? ? ? ? 307 1.0
*SOURCE ? ? 365 ? ?
*TARGET ? ? ? ? 365 1.0
*SOURCE ? ? 357 ? ?
*TARGET ? ? ? ? 357 1.0
*SOURCE ? ? 733 ? ?
*TARGET ? ? ? ? 733 1.0
*SOURCE ? ? 265 ? ?
*TARGET ? ? ? ? 265 1.0
*SOURCE ? ? 387 ? ?
*TARGET ? ? ? ? 387 1.0
*SOURCE ? ? 710 ? ?
*TARGET ? ? ? ? 710 1.0
*SOURCE ? ? 265 ? ?
*TARGET ? ? ? ? 265 1.0
*SOURCE ? ? 306 ? ?
*TARGET ? ? ? ? 306 1.0
*SOURCE ? ? 711 ? ?
*TARGET ? ? ? ? 711 1.0
*SOURCE ? ? 733 ? ?
*TARGET ? ? ? ? 733 1.0
*SOURCE ? ? 298 ? ?
*TARGET ? ? ? ? 298 1.0
*SOURCE ? ? 357 ? ?
*TARGET ? ? ? ? 357 1.0
*SOURCE ? ? 347 ? ?
*TARGET ? ? ? ? 347 1.0
*SOURCE ? ? 357 ? ?
*TARGET ? ? ? ? 357 1.0
*SOURCE ? ? 378 ? ?
*TARGET ? ? ? ? 378 1.0
*SOURCE ? ? 379 ? ?
*TARGET ? ? ? ? 379 1.0
*SOURCE ? ? 336 ? ?
*TARGET ? ? ? ? 336 1.0
*SOURCE ? ? 378 ? ?
*TARGET ? ? ? ? 378 1.0
*SOURCE ? ? 734 ? ?
*TARGET ? ? ? ? 734 1.0
*SOURCE ? ? 378 ? ?
*TARGET ? ? ? ? 378 1.0
*SOURCE ? ? 733 ? ?
*TARGET ? ? ? ? 733 1.0
*SOURCE ? ? 682 ? ?
*TARGET ? ? ? ? 682 1.0
*SOURCE ? ? 357 ? ?
*TARGET ? ? ? ? 357 1.0
*SOURCE ? ? 717 ? ?
*TARGET ? ? ? ? 717 1.0
*SOURCE ? ? 357 ? ?
*TARGET ? ? ? ? 357 1.0
*SOURCE ? ? 717 ? ?
*TARGET ? ? ? ? 717 1.0
*SOURCE ? ? 734 ? ?
*TARGET ? ? ? ? 734 1.0
*SOURCE ? ? 356 ? ?
*TARGET ? ? ? ? 356 1.0
*SOURCE ? ? 733 ? ?
*TARGET ? ? ? ? 733 1.0
*SOURCE ? ? 733 ? ?
*TARGET ? ? ? ? 733 1.0
*SOURCE ? ? 1192 ? ?
*TARGET ? ? ? ? 1192 1.0
*SOURCE ? ? 733 ? ?
*TARGET ? ? ? ? 733 1.0
*SOURCE ? ? 733 ? ?
*TARGET ? ? ? ? 733 1.0
*SOURCE ? ? 734 ? ?
*TARGET ? ? ? ? 734 1.0
*SOURCE ? ? 379 ? ?
*TARGET ? ? ? ? 379 1.0
*SOURCE ? ? 682 ? ?
*TARGET ? ? ? ? 682 1.0
*SOURCE ? ? 734 ? ?
*TARGET ? ? ? ? 734 1.0
*SOURCE ? ? 734 ? ?
*TARGET ? ? ? ? 734 1.0
*SOURCE ? ? 307 ? ?
*TARGET ? ? ? ? 307 1.0
*SOURCE ? ? 388 ? ?
*TARGET ? ? ? ? 388 1.0
*SOURCE ? ? 366 ? ?
*TARGET ? ? ? ? 366 1.0
*SOURCE ? ? 734 ? ?
*TARGET ? ? ? ? 734 1.0
*SOURCE ? ? 733 ? ?
*TARGET ? ? ? ? 733 1.0
*SOURCE ? ? 733 ? ?
*TARGET ? ? ? ? 733 1.0
*SOURCE ? ? 733 ? ?
*TARGET ? ? ? ? 733 1.0
*SOURCE ? ? 733 ? ?
*TARGET ? ? ? ? 733 1.0
*SOURCE ? ? 717 ? ?
*TARGET ? ? ? ? 717 1.0
*SOURCE ? ? 734 ? ?
*TARGET ? ? ? ? 734 1.0
*SOURCE ? ? 336 ? ?
*TARGET ? ? ? ? 336 1.0
*SOURCE ? ? 718 ? ?
*TARGET ? ? ? ? 718 1.0
*SOURCE ? ? 733 ? ?
*TARGET ? ? ? ? 733 1.0
*SOURCE ? ? 387 ? ?
*TARGET ? ? ? ? 387 1.0
*SOURCE ? ? 306 ? ?
*TARGET ? ? ? ? 306 1.0
*SOURCE ? ? 734 ? ?
*TARGET ? ? ? ? 734 1.0
*SOURCE ? ? 734 ? ?
*TARGET ? ? ? ? 734 1.0
*SOURCE ? ? 365 ? ?
*TARGET ? ? ? ? 365 1.0
*SOURCE ? ? 734 ? ?
*TARGET ? ? ? ? 734 1.0
*SOURCE ? ? 365 ? ?
*TARGET ? ? ? ? 365 1.0
*SOURCE ? ? 734 ? ?
*TARGET ? ? ? ? 734 1.0
*SOURCE ? ? 734 ? ?
*TARGET ? ? ? ? 734 1.0
*SOURCE ? ? 734 ? ?
*TARGET ? ? ? ? 734 1.0
*SOURCE ? ? 734 ? ?
*TARGET ? ? ? ? 734 1.0
*SOURCE ? ? 388 ? ?
*TARGET ? ? ? ? 388 1.0
*SOURCE ? ? 733 ? ?
*TARGET ? ? ? ? 733 1.0
*SOURCE ? ? 265 ? ?
*TARGET ? ? ? ? 265 1.0
*SOURCE ? ? 365 ? ?
*TARGET ? ? ? ? 365 1.0
*SOURCE ? ? 387 ? ?
*TARGET ? ? ? ? 387 1.0
*SOURCE ? ? 357 ? ?
*TARGET ? ? ? ? 357 1.0
*SOURCE ? ? 265 ? ?
*TARGET ? ? ? ? 265 1.0
*SOURCE ? ? 356 ? ?
*TARGET ? ? ? ? 356 1.0
*SOURCE ? ? 733 ? ?
*TARGET ? ? ? ? 733 1.0
*SOURCE ? ? 682 ? ?
*TARGET ? ? ? ? 682 1.0
*SOURCE ? ? 365 ? ?
*TARGET ? ? ? ? 365 1.0
*SOURCE ? ? 682 ? ?
*TARGET ? ? ? ? 682 1.0
*SOURCE ? ? 733 ? ?
*TARGET ? ? ? ? 733 1.0
*SOURCE ? ? 734 ? ?
*TARGET ? ? ? ? 734 1.0
*SOURCE ? ? 348 ? ?
*TARGET ? ? ? ? 348 1.0
*SOURCE ? ? 733 ? ?
*TARGET ? ? ? ? 733 1.0
*SOURCE ? ? 306 ? ?
*TARGET ? ? ? ? 306 1.0
*SOURCE ? ? 366 ? ?
*TARGET ? ? ? ? 366 1.0
*SOURCE ? ? 357 ? ?
*TARGET ? ? ? ? 357 1.0
*SOURCE ? ? 387 ? ?
*TARGET ? ? ? ? 387 1.0
*SOURCE ? ? 690 ? ?
*TARGET ? ? ? ? 690 1.0
*SOURCE ? ? 365 ? ?
*TARGET ? ? ? ? 365 1.0
*SOURCE ? ? 365 ? ?
*TARGET ? ? ? ? 365 1.0
*SOURCE ? ? 733 ? ?
*TARGET ? ? ? ? 733 1.0
*SOURCE ? ? 734 ? ?
*TARGET ? ? ? ? 734 1.0
*SOURCE ? ? 336 ? ?
*TARGET ? ? ? ? 336 1.0
*SOURCE ? ? 690 ? ?
*TARGET ? ? ? ? 690 1.0
*SOURCE ? ? 387 ? ?
*TARGET ? ? ? ? 387 1.0
*SOURCE ? ? 265 ? ?
*TARGET ? ? ? ? 265 1.0
*SOURCE ? ? 733 ? ?
*TARGET ? ? ? ? 733 1.0
*SOURCE ? ? 735 ? ?
*TARGET ? ? ? ? 735 1.0
*SOURCE ? ? 734 ? ?
*TARGET ? ? ? ? 734 1.0
*SOURCE ? ? 306 ? ?
*TARGET ? ? ? ? 306 1.0
*SOURCE ? ? 265 ? ?
*TARGET ? ? ? ? 265 1.0
*SOURCE ? ? 312 ? ?
*TARGET ? ? ? ? 312 1.0
*SOURCE ? ? 312 ? ?
*TARGET ? ? ? ? 312 1.0
*SOURCE ? ? 265 ? ?
*TARGET ? ? ? ? 265 1.0
*SOURCE ? ? 265 ? ?
*TARGET ? ? ? ? 265 1.0
*SOURCE ? ? 312 ? ?
*TARGET ? ? ? ? 312 1.0
*SOURCE ? ? 306 ? ?
*TARGET ? ? ? ? 306 1.0
*SOURCE ? ? 265 ? ?
*TARGET ? ? ? ? 265 1.0
*SOURCE ? ? 265 ? ?
*TARGET ? ? ? ? 265 1.0
*SOURCE ? ? 265 ? ?
*TARGET ? ? ? ? 265 1.0
*SOURCE ? ? 306 ? ?
*TARGET ? ? ? ? 306 1.0
*SOURCE ? ? 265 ? ?
*TARGET ? ? ? ? 265 1.0
*SOURCE ? ? 265 ? ?
*TARGET ? ? ? ? 265 1.0
*SOURCE ? ? 265 ? ?
*TARGET ? ? ? ? 265 1.0
*SOURCE ? ? 306 ? ?
*TARGET ? ? ? ? 306 1.0
*SOURCE ? ? 387 ? ?
*TARGET ? ? ? ? 387 1.0
*SOURCE ? ? 387 ? ?
*TARGET ? ? ? ? 387 1.0
*SOURCE ? ? 335 ? ?
*TARGET ? ? ? ? 335 1.0
*SOURCE ? ? 336 ? ?
*TARGET ? ? ? ? 336 1.0
*SOURCE ? ? 365 ? ?
*TARGET ? ? ? ? 365 1.0
*SOURCE ? ? 734 ? ?
*TARGET ? ? ? ? 734 1.0
*SOURCE ? ? 357 ? ?
*TARGET ? ? ? ? 357 1.0
*SOURCE ? ? 365 ? ?
*TARGET ? ? ? ? 365 1.0
*SOURCE ? ? 379 ? ?
*TARGET ? ? ? ? 379 1.0
*SOURCE ? ? 734 ? ?
*TARGET ? ? ? ? 734 1.0
*SOURCE ? ? 335 ? ?
*TARGET ? ? ? ? 335 1.0
*SOURCE ? ? 690 ? ?
*TARGET ? ? ? ? 690 1.0
*SOURCE ? ? 336 ? ?
*TARGET ? ? ? ? 336 1.0
*SOURCE ? ? 728 ? ?
*TARGET ? ? ? ? 728 1.0
*SOURCE ? ? 357 ? ?
*TARGET ? ? ? ? 357 1.0
*SOURCE ? ? 344 ? ?
*TARGET ? ? ? ? 344 1.0
*SOURCE ? ? 728 ? ?
*TARGET ? ? ? ? 728 1.0
*SOURCE ? ? 728 ? ?
*TARGET ? ? ? ? 728 1.0
*SOURCE ? ? 728 ? ?
*TARGET ? ? ? ? 728 1.0
*SOURCE ? ? 734 ? ?
*TARGET ? ? ? ? 734 1.0
*SOURCE ? ? 335 ? ?
*TARGET ? ? ? ? 335 1.0
*SOURCE ? ? 378 ? ?
*TARGET ? ? ? ? 378 1.0
*SOURCE ? ? 379 ? ?
*TARGET ? ? ? ? 379 1.0
*SOURCE ? ? 344 ? ?
*TARGET ? ? ? ? 344 1.0
*SOURCE ? ? 379 ? ?
*TARGET ? ? ? ? 379 1.0
*SOURCE ? ? 365 ? ?
*TARGET ? ? ? ? 365 1.0
*SOURCE ? ? 388 ? ?
*TARGET ? ? ? ? 388 1.0
*SOURCE ? ? 353 ? ?
*TARGET ? ? ? ? 353 1.0
*SOURCE ? ? 356 ? ?
*TARGET ? ? ? ? 356 1.0
*SOURCE ? ? 379 ? ?
*TARGET ? ? ? ? 379 1.0
*SOURCE ? ? 298 ? ?
*TARGET ? ? ? ? 298 1.0
*SOURCE ? ? 265 ? ?
*TARGET ? ? ? ? 265 1.0
*SOURCE ? ? 366 ? ?
*TARGET ? ? ? ? 366 1.0
*SOURCE ? ? 344 ? ?
*TARGET ? ? ? ? 344 1.0
*SOURCE ? ? 357 ? ?
*TARGET ? ? ? ? 357 1.0
*SOURCE ? ? 378 ? ?
*TARGET ? ? ? ? 378 1.0
*SOURCE ? ? 265 ? ?
*TARGET ? ? ? ? 265 1.0
*SOURCE ? ? 299 ? ?
*TARGET ? ? ? ? 299 1.0
*SOURCE ? ? 344 ? ?
*TARGET ? ? ? ? 344 1.0
*SOURCE ? ? 306 ? ?
*TARGET ? ? ? ? 306 1.0
*SOURCE ? ? 318 ? ?
*TARGET ? ? ? ? 318 1.0
*SOURCE ? ? 378 ? ?
*TARGET ? ? ? ? 378 1.0
*SOURCE ? ? 298 ? ?
*TARGET ? ? ? ? 298 1.0
*SOURCE ? ? 307 ? ?
*TARGET ? ? ? ? 307 1.0
*SOURCE ? ? 298 ? ?
*TARGET ? ? ? ? 298 1.0
*SOURCE ? ? 388 ? ?
*TARGET ? ? ? ? 388 1.0
*SOURCE ? ? 378 ? ?
*TARGET ? ? ? ? 378 1.0
*SOURCE ? ? 388 ? ?
*TARGET ? ? ? ? 388 1.0
*SOURCE ? ? 344 ? ?
*TARGET ? ? ? ? 344 1.0
*SOURCE ? ? 344 ? ?
*TARGET ? ? ? ? 344 1.0
*SOURCE ? ? 379 ? ?
*TARGET ? ? ? ? 379 1.0
*SOURCE ? ? 388 ? ?
*TARGET ? ? ? ? 388 1.0
*SOURCE ? ? 298 ? ?
*TARGET ? ? ? ? 298 1.0
*SOURCE ? ? 400 ? ?
*TARGET ? ? ? ? 400 1.0
*SOURCE ? ? 400 ? ?
*TARGET ? ? ? ? 400 1.0
*SOURCE ? ? 296 ? ?
*TARGET ? ? ? ? 296 1.0
*SOURCE ? ? 388 ? ?
*TARGET ? ? ? ? 388 1.0
*SOURCE ? ? 387 ? ?
*TARGET ? ? ? ? 387 1.0
*SOURCE ? ? 387 ? ?
*TARGET ? ? ? ? 387 1.0
*SOURCE ? ? 388 ? ?
*TARGET ? ? ? ? 388 1.0
*SOURCE ? ? 307 ? ?
*TARGET ? ? ? ? 307 1.0
*SOURCE ? ? 387 ? ?
*TARGET ? ? ? ? 387 1.0
*SOURCE ? ? 265 ? ?
*TARGET ? ? ? ? 265 1.0
*SOURCE ? ? 379 ? ?
*TARGET ? ? ? ? 379 1.0
*SOURCE ? ? 387 ? ?
*TARGET ? ? ? ? 387 1.0
*SOURCE ? ? 307 ? ?
*TARGET ? ? ? ? 307 1.0
*SOURCE ? ? 307 ? ?
*TARGET ? ? ? ? 307 1.0
*SOURCE ? ? 266 ? ?
*TARGET ? ? ? ? 266 1.0
*SOURCE ? ? 265 ? ?
*TARGET ? ? ? ? 265 1.0
*SOURCE ? ? 379 ? ?
*TARGET ? ? ? ? 379 1.0
*SOURCE ? ? 266 ? ?
*TARGET ? ? ? ? 266 1.0
*SOURCE ? ? 388 ? ?
*TARGET ? ? ? ? 388 1.0
*SOURCE ? ? 388 ? ?
*TARGET ? ? ? ? 388 1.0
*SOURCE ? ? 257 ? ?
*TARGET ? ? ? ? 257 1.0
*SOURCE ? ? 388 ? ?
*TARGET ? ? ? ? 388 1.0
*SOURCE ? ? 265 ? ?
*TARGET ? ? ? ? 265 1.0
*SOURCE ? ? 336 ? ?
*TARGET ? ? ? ? 336 1.0
*SOURCE ? ? 299 ? ?
*TARGET ? ? ? ? 299 1.0
*SOURCE ? ? 388 ? ?
*TARGET ? ? ? ? 388 1.0
*SOURCE ? ? 366 ? ?
*TARGET ? ? ? ? 366 1.0
*SOURCE ? ? 299 ? ?
*TARGET ? ? ? ? 299 1.0
*SOURCE ? ? 388 ? ?
*TARGET ? ? ? ? 388 1.0
*SOURCE ? ? 336 ? ?
*TARGET ? ? ? ? 336 1.0
*SOURCE ? ? 379 ? ?
*TARGET ? ? ? ? 379 1.0
*SOURCE ? ? 306 ? ?
*TARGET ? ? ? ? 306 1.0
*SOURCE ? ? 387 ? ?
*TARGET ? ? ? ? 387 1.0
*SOURCE ? ? 378 ? ?
*TARGET ? ? ? ? 378 1.0
*SOURCE ? ? 357 ? ?
*TARGET ? ? ? ? 357 1.0
*SOURCE ? ? 357 ? ?
*TARGET ? ? ? ? 357 1.0
*SOURCE ? ? 336 ? ?
*TARGET ? ? ? ? 336 1.0
*SOURCE ? ? 344 ? ?
*TARGET ? ? ? ? 344 1.0
*SOURCE ? ? 378 ? ?
*TARGET ? ? ? ? 378 1.0
*SOURCE ? ? 299 ? ?
*TARGET ? ? ? ? 299 1.0
*SOURCE ? ? 357 ? ?
*TARGET ? ? ? ? 357 1.0
*SOURCE ? ? 306 ? ?
*TARGET ? ? ? ? 306 1.0
*SOURCE ? ? 357 ? ?
*TARGET ? ? ? ? 357 1.0
*SOURCE ? ? 307 ? ?
*TARGET ? ? ? ? 307 1.0
*SOURCE ? ? 379 ? ?
*TARGET ? ? ? ? 379 1.0
*SOURCE ? ? 357 ? ?
*TARGET ? ? ? ? 357 1.0
*SOURCE ? ? 378 ? ?
*TARGET ? ? ? ? 378 1.0
*SOURCE ? ? 357 ? ?
*TARGET ? ? ? ? 357 1.0
*SOURCE ? ? 379 ? ?
*TARGET ? ? ? ? 379 1.0
*SOURCE ? ? 307 ? ?
*TARGET ? ? ? ? 307 1.0
*SOURCE ? ? 357 ? ?
*TARGET ? ? ? ? 357 1.0
*SOURCE ? ? 379 ? ?
*TARGET ? ? ? ? 379 1.0
*SOURCE ? ? 336 ? ?
*TARGET ? ? ? ? 336 1.0
*SOURCE ? ? 379 ? ?
*TARGET ? ? ? ? 379 1.0
*SOURCE ? ? 357 ? ?
*TARGET ? ? ? ? 357 1.0
*SOURCE ? ? 357 ? ?
*TARGET ? ? ? ? 357 1.0
*SOURCE ? ? 357 ? ?
*TARGET ? ? ? ? 357 1.0
*SOURCE ? ? 299 ? ?
*TARGET ? ? ? ? 299 1.0
*SOURCE ? ? 379 ? ?
*TARGET ? ? ? ? 379 1.0
*SOURCE ? ? 336 ? ?
*TARGET ? ? ? ? 336 1.0
*SOURCE ? ? 307 ? ?
*TARGET ? ? ? ? 307 1.0
*SOURCE ? ? 307 ? ?
*TARGET ? ? ? ? 307 1.0
*SOURCE ? ? 379 ? ?
*TARGET ? ? ? ? 379 1.0
*SOURCE ? ? 336 ? ?
*TARGET ? ? ? ? 336 1.0
*SOURCE ? ? 388 ? ?
*TARGET ? ? ? ? 388 1.0
*SOURCE ? ? 378 ? ?
*TARGET ? ? ? ? 378 1.0
*SOURCE ? ? 379 ? ?
*TARGET ? ? ? ? 379 1.0
*SOURCE ? ? 379 ? ?
*TARGET ? ? ? ? 379 1.0
*SOURCE ? ? 357 ? ?
*TARGET ? ? ? ? 357 1.0
*SOURCE ? ? 298 ? ?
*TARGET ? ? ? ? 298 1.0
*SOURCE ? ? 298 ? ?
*TARGET ? ? ? ? 298 1.0
*SOURCE ? ? 378 ? ?
*TARGET ? ? ? ? 378 1.0
*SOURCE ? ? 378 ? ?
*TARGET ? ? ? ? 378 1.0
*SOURCE ? ? 299 ? ?
*TARGET ? ? ? ? 299 1.0
*SOURCE ? ? 379 ? ?
*TARGET ? ? ? ? 379 1.0
*SOURCE ? ? 733 ? ?
*TARGET ? ? ? ? 733 1.0
*SOURCE ? ? 335 ? ?
*TARGET ? ? ? ? 335 1.0
*SOURCE ? ? 356 ? ?
*TARGET ? ? ? ? 356 1.0
*SOURCE ? ? 379 ? ?
*TARGET ? ? ? ? 379 1.0
*SOURCE ? ? 711 ? ?
*TARGET ? ? ? ? 711 1.0
*SOURCE ? ? 379 ? ?
*TARGET ? ? ? ? 379 1.0
*SOURCE ? ? 379 ? ?
*TARGET ? ? ? ? 379 1.0
*SOURCE ? ? 357 ? ?
*TARGET ? ? ? ? 357 1.0
*SOURCE ? ? 379 ? ?
*TARGET ? ? ? ? 379 1.0
*SOURCE ? ? 299 ? ?
*TARGET ? ? ? ? 299 1.0
*SOURCE ? ? 344 ? ?
*TARGET ? ? ? ? 344 1.0
*SOURCE ? ? 379 ? ?
*TARGET ? ? ? ? 379 1.0
*SOURCE ? ? 379 ? ?
*TARGET ? ? ? ? 379 1.0
*SOURCE ? ? 357 ? ?
*TARGET ? ? ? ? 357 1.0
*SOURCE ? ? 307 ? ?
*TARGET ? ? ? ? 307 1.0
*SOURCE ? ? 336 ? ?
*TARGET ? ? ? ? 336 1.0
*SOURCE ? ? 277 ? ?
*TARGET ? ? ? ? 277 1.0
*SOURCE ? ? 379 ? ?
*TARGET ? ? ? ? 379 1.0
*SOURCE ? ? 378 ? ?
*TARGET ? ? ? ? 378 1.0
*SOURCE ? ? 357 ? ?
*TARGET ? ? ? ? 357 1.0
*SOURCE ? ? 378 ? ?
*TARGET ? ? ? ? 378 1.0
*SOURCE ? ? 378 ? ?
*TARGET ? ? ? ? 378 1.0
*SOURCE ? ? 298 ? ?
*TARGET ? ? ? ? 298 1.0
*SOURCE ? ? 356 ? ?
*TARGET ? ? ? ? 356 1.0
*SOURCE ? ? 682 ? ?
*TARGET ? ? ? ? 682 1.0
*SOURCE ? ? 378 ? ?
*TARGET ? ? ? ? 378 1.0
*SOURCE ? ? 711 ? ?
*TARGET ? ? ? ? 711 1.0
*SOURCE ? ? 733 ? ?
*TARGET ? ? ? ? 733 1.0
*SOURCE ? ? 378 ? ?
*TARGET ? ? ? ? 378 1.0
*SOURCE ? ? 356 ? ?
*TARGET ? ? ? ? 356 1.0
*SOURCE ? ? 734 ? ?
*TARGET ? ? ? ? 734 1.0
*SOURCE ? ? 344 ? ?
*TARGET ? ? ? ? 344 1.0
*SOURCE ? ? 298 ? ?
*TARGET ? ? ? ? 298 1.0
*SOURCE ? ? 387 ? ?
*TARGET ? ? ? ? 387 1.0
*SOURCE ? ? 378 ? ?
*TARGET ? ? ? ? 378 1.0
*SOURCE ? ? 307 ? ?
*TARGET ? ? ? ? 307 1.0
*SOURCE ? ? 379 ? ?
*TARGET ? ? ? ? 379 1.0
*SOURCE ? ? 1103 ? ?
*TARGET ? ? ? ? 1103 1.0
*SOURCE ? ? 734 ? ?
*TARGET ? ? ? ? 734 1.0
*SOURCE ? ? 734 ? ?
*TARGET ? ? ? ? 734 1.0
*SOURCE ? ? 379 ? ?
*TARGET ? ? ? ? 379 1.0
*SOURCE ? ? 379 ? ?
*TARGET ? ? ? ? 379 1.0
*SOURCE ? ? 336 ? ?
*TARGET ? ? ? ? 336 1.0
*SOURCE ? ? 357 ? ?
*TARGET ? ? ? ? 357 1.0
*SOURCE ? ? 378 ? ?
*TARGET ? ? ? ? 378 1.0
*SOURCE ? ? 357 ? ?
*TARGET ? ? ? ? 357 1.0
*SOURCE ? ? 388 ? ?
*TARGET ? ? ? ? 388 1.0
*SOURCE ? ? 379 ? ?
*TARGET ? ? ? ? 379 1.0
*SOURCE ? ? 387 ? ?
*TARGET ? ? ? ? 387 1.0
*SOURCE ? ? 265 ? ?
*TARGET ? ? ? ? 265 1.0
*SOURCE ? ? 317 ? ?
*TARGET ? ? ? ? 317 1.0
*SOURCE ? ? 265 ? ?
*TARGET ? ? ? ? 265 1.0
*SOURCE ? ? 306 ? ?
*TARGET ? ? ? ? 306 1.0
"""

In [ ]:
with open('data/woodstock_model_files/tsa04.trn', 'w') as f: f.write(transitions_section)

# Create and run `ForestModel` instance

In [ ]:
fm = ws3.forest.ForestModel(model_name='tsa04',
                            model_path='data/woodstock_model_files',
                            base_year=base_year,
                            horizon=horizon,
                            period_length=period_length,
                            max_age=max_age)

In [ ]:
fm.import_landscape_section()

In [ ]:
fm.import_areas_section()

In [ ]:
fm.import_yields_section()

In [ ]:
fm.import_actions_section()

In [ ]:
fm.import_transitions_section()

In [ ]:
fm.initialize_areas()
fm.reset_actions()

In [ ]:
# copy function definition for debugging
def schedule_harvest_areacontrol(fm, period=1, acode='harvest', util=0.85, 
                                 target_masks=None, target_areas=None, target_scalefactors=None,
                                 mask_area_thresh=0.,
                                 verbose=0):
    #fm.reset_actions()
    if not target_areas:
        if not target_masks: # default to AU-wise THLB 
            au_vals = []
            au_agg = []
            for au in fm.theme_basecodes(2):
                mask = '? 1 %s ? ?' % au
                masked_area = fm.inventory(0, mask=mask)
                #print(mask, masked_area)
                if masked_area > mask_area_thresh:
                    #print(masked_area, mask_area_thresh)
                    au_vals.append(au)
                else:
                    au_agg.append(au)
                    if verbose > 0:
                        print('adding to au_agg', mask, masked_area)
                #print(au_vals)
            if au_agg:
                fm._themes[2]['areacontrol_au_agg'] = au_agg 
                if fm.inventory(0, mask='? ? areacontrol_au_agg ? ?') > mask_area_thresh:
                    au_vals.append('areacontrol_au_agg')
            target_masks = ['? 1 %s ? ?' % au for au in au_vals]
            #print(au_vals)
            #print(target_masks)
        print(target_masks, au_vals)
        #assert False
        target_areas = []
        for i, mask in enumerate(target_masks): # compute area-weighted mean CMAI age for each masked DT set
            print(mask)
            masked_area = fm.inventory(0, mask=mask, verbose=verbose)
            print(masked_area)
            if not masked_area: continue
            r = sum((fm.dtypes[dtk].ycomp('totvol').mai().ytp().lookup(0) * fm.dtypes[dtk].area(0)) for dtk in fm.unmask(mask))
            r /= masked_area
            #awr = []
            #dtype_keys = fm.unmask(mask)
            #for dtk in dtype_keys:
            #    dt = fm.dtypes[dtk]
            #    awr.append(dt.ycomp('totvol').mai().ytp().lookup(0) * dt.area(0))
            #r = sum(awr)  / masked_area
            asf = 1. if not target_scalefactors else target_scalefactors[i]  
            # print(i)
            # print(mask)
            # print(masked_area)
            ta = (1/r) * fm.period_length * masked_area * asf
            target_areas.append(ta)
    for mask, target_area in zip(target_masks, target_areas):
        if verbose > 0:
            print('calling areaselector', period, acode, target_area, mask)
        fm.areaselector.operate(period, acode, target_area, mask=mask, verbose=verbose)
    sch = fm.compile_schedule()
    return sch

In [ ]:
schedule_harvest_areacontrol(fm, period=1, verbose=1)

In [ ]:
for period in fm.periods:
    schedule_harvest_areacontrol(fm, period=period, verbose=verbose)

In [ ]:
pareas = [fm.compile_product(period, '1.') for period in fm.periods]

In [ ]:
pvols = [fm.compile_product(period, 'totvol') for period in fm.periods]

In [ ]:
df = pd.DataFrame({'period':fm.periods, 'areas':pareas, 'vols':pvols})
df

In [ ]:
(df.vols/df.areas).plot()